In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import shutil
import json
import pickle
from tqdm import tqdm
import random
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt
from PIL import Image

from load_model_ver3 import load_model
from load_model_barlowtwins import load_barlowtwins_model

from rn50_auxiliary_dm import rn50_auxiliary_dm

import scipy

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [ ]:
from load_model import load_model
from rn50_auxiliary_dm import rn50_auxiliary_dm

In [ ]:
import scipy

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import svm

# extract feats

## extract feats

In [ ]:
# Experiment list summarized

exp_names_list_textures = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_color_texture_resized/id0',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_color_texture_resized/id1',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_gray_downsampled/camel_colored_gray',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_gray_downsampled/elephant_colored_gray',
    
    # '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_downsampled/camel_colored',
    # '/mnt/smb/locker/issa-locker/users/Seojin/data/object_data/GOR_colored_downsampled/elephant_colored',
]

exp_names_list_basics = [
    # 1. Josh - Elias/Neptune neutral
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_elias',
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_neptune',

    # 2. Seojin - Elias/Neptune neutral
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_neptune/vbsle151_neptune_neutral',

    # 3a. Josh - Sophie/Praneeth
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_sophie_praneeth/20230406_Var6vbsl_set0_im151_praneeth_dur200ms_lab',
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_sophie_praneeth/20230406_Var6vbsl_set0_im151_sophie_dur200ms_lab',

    # 3b. Josh - Basel - texture
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood/20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_dur200ms_lab',
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood/20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_dur200ms_lab',

    # 3c.Josh - Basel - no texture
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_notexture_dur200ms_lab',
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_notexture_dur200ms_lab',

    # 3d. Josh - Camel/Elephant
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_camel',
    '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant',

    # 3e.     
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_dog_horse/vbsle_151_dog', 
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_dog_horse/vbsle_151_horse',

    # textured elais/neptune
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/textured_elias_grayscaled', 
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/textured_neptune_grayscaled',

]

exp_names_list_imagenet = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/shark151/shark1',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/shark151/shark2',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/bird151/bird1',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/bird151/bird2',
    
    '/mnt/smb/locker/issa-locker/users/Seojin/data/vehicle151/bike',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/vehicle151/car'
]

exp_names_list_GFR = [
    # 1. Elias - Neptune
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_neptune/vbsle151_neptune_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_seojin/vbsle151_seojin_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_dan/vbsle151_dan_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sreyas/vbsle151_sreyas_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_younah/vbsle151_younah_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_ashley/vbsle151_ashley_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_kedar/vbsle151_kedar_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_josh/vbsle151_josh_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_ani/vbsle151_ani_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_tony/vbsle151_tony_neutral',
]

exp_names_list_GFR_texture_colorbg = [
    # 1. Elias - Neptune
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_neptune_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_seojin_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_sophie_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_dan_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_sreyas_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_younah_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_ashley_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_kedar_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_josh_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_ani_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_tony_neutral',
]
exp_names_list_GFR_texture_graybg = [
    # 1. Elias - Neptune
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_neptune_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_seojin_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_sophie_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_dan_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_sreyas_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_younah_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_ashley_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_kedar_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_josh_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_ani_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_tony_neutral',
]

exp_names_list_GER_sophie = [ # Sophie emotion 4
    # 1. n-h
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_disgust_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_sophie/vbsle151_sophie_surprise_4',
   ]

exp_names_list_GER_elias  = [ # elias emotion 4
    # 1. n-h
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_disgust_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/vbsle151_elias_surprise_4',
    ]

exp_names_list_GER_elias_texture_colorbg = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_disgust_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/vbsl151_elias_surprise_4',
]

exp_names_list_GER_elias_texture_graybg = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_neutral',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_happiness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_sadness_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_disgust_4'
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_fear_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_anger_4',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_graybg/vbsl151_elias_surprise_4',
]

exp_names_list_lfw = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell/Bush',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell/Powell',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell_inverted/Bush',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/LFW_Bush_Powell_inverted/Powell',
]

exp_names_list_ytface = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Stanca',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Beard',

    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Kennedy',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/YoutubeFaces/Macdonald',
]


In [ ]:
exp_names_list_basel = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id10',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id11',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id12',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id13',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id14',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id15',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id16',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id17',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id18',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id19',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id20',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/id21',
]

exp_names_list_basel_texture_colorbg = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id10',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id11',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id12',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id13',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id14',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id15',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id16',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id17',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id18',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id19',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id20',
'/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/id21',
]

In [ ]:
import gc
import torch 
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Standard models 
exp_names_list = exp_names_list_GFR

def update_state_dict_keys(state_dict):
    new_state_dict = {}
    for k, v in state_dict.items():
        # print(k)
        # if k.startswith('module.shared_layers.'):
        #     new_key = k.replace('module.shared_layers.', 'module.')
        # print(k)
        if k.startswith('module.'):
            new_key = k.replace('module.', 'module.module.')
            # print(new_key)
        # else:
        #     new_key = 'module.' + k if not k.startswith('module.') else k
        new_state_dict[new_key] = v
        # print("new", new_key)
    return new_state_dict

def adjust_fc_layer(model, state_dict):
    for key in state_dict.keys():
        if 'fc.weight' in key:
            final_layer_key = key
            break
    num_classes = state_dict[final_layer_key].shape[0]
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


# Pick the experiment set
# model_names_list=[
# # 'off_the_shelf_barlowtwins_finetune_7way_EM_sophie_colorbg_seed77_model_best',
# # 'off_the_shelf_barlowtwins_finetune_7way_EM_dan_colorbg_seed77__model_best',
# # 'off_the_shelf_barlowtwins_finetune_7way_EM_kedar_colorbg_seed77__model_best',
# 'off_the_shelf_barlowtwins_finetune_4way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_4way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_4way_IDEM_sophie_kedar_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_6way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_6way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_6way_IDEM_sophie_kedar_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_8way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_8way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_8way_IDEM_sophie_kedar_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_10way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_10way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_10way_IDEM_sophie_kedar_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_12way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_12way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'off_the_shelf_barlowtwins_finetune_12way_IDEM_sophie_kedar_colorbg_seed77_model_best',
# ]
# model_names_list = [
#     # 'off_the_shelf_barlowtwins_seojin_dan_epochs50',
#     'off_the_shelf_barlowtwins_2way_seojin_sophie',
#     'off_the_shelf_barlowtwins_2way_sophie_kedar',
#     'off_the_shelf_barlowtwins_finetune_4way_SSKD',
#     # 'off_the_shelf_barlowtwins_finetune_6way_final',
#     # 'off_the_shelf_barlowtwins_ID_8way_epochs50',

#     'off_the_shelf_barlowtwins_texture_colorbg_2way_seojin_dan',
#     'off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_seojin', 
#     'off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_kedar',
#     # 'off_the_shelf_barlowtwins_texture_colorbg_4way',
#     # 'off_the_shelf_barlowtwins_texture_colorbg_6way',
#     # 'off_the_shelf_barlowtwins_ID_8way_colorbg_epochs50',

#     # 'off_the_shelf_barlowtwins_finetune_texture_colorbg_em_neutral_anger',
#     # 'off_the_shelf_barlowtwins_finetune_texture_colorbg_em_4way_NHAS',
#     # 'off_the_shelf_barlowtwins_finetune_texture_colorbg_em_6way_excl_fear'
# ]
model_names_list = ['barlowtwins_finetune_7way_EM_seojin_seed77__model_best'
                    ]
filename_postfix = ''
save_label = False

for model_name in model_names_list:
    # flatten & subsample for previous layers of resnet50
    if 'layer' in model_name or "subsampled" in model_name:
        if "layerC2" in model_name and "layerC2_subsampled" not in model_name:
            subsample = False
        else:
            subsample = True
    else:
        subsample = False
    
    # load model
    model = load_model(model_name)
    model.eval()
    
    # add hook
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    if 'simplecnn' in model_name:
        model.fc2.register_forward_hook(get_activation('feats'))
    elif 'alexnet' in model_name:
        if 'layer4' in model_name:
            print("add hook in layer4")
            model.features[4].register_forward_hook(get_activation('feats'))
        elif 'layer8' in model_name:
            print("add hook in layer8")
            model.features[8].register_forward_hook(get_activation('feats'))
        elif 'layer12' in model_name:
            print("add hook in layer12")
            model.features[12].register_forward_hook(get_activation('feats'))
        elif 'layerC2' in model_name:
            model.classifier[2].register_forward_hook(get_activation('feats'))
        else: # final layer
            model.classifier[5].register_forward_hook(get_activation('feats'))
    elif model_name == 'vgg16':
        model.classifier[4].register_forward_hook(get_activation('feats'))
    elif model_name == 'vggface':
        model.fc7.register_forward_hook(get_activation('feats'))
    elif 'onlyDM' in model_name:
        model.depth_predictor.encoder[4].register_forward_hook(get_activation('feats'))
        # model.depth_predictor.decoder[30].register_forward_hook(get_activation('feats'))
        # model.depth_predictor.encoder[6].register_forward_hook(get_activation('feats'))
    elif 'head' in model_name :
        model.module.backbone.avgpool.register_forward_hook(get_activation('feats'))
    elif 'remapping' in model_name :
        model.module.backbone.avgpool.register_forward_hook(get_activation('feats'))
    else: # resnet50
        if 'layer1' in model_name:
            print("add hook in layer1")
            model.layer1.register_forward_hook(get_activation('feats'))
        elif 'layer2' in model_name:
            print("add hook in layer2")
            model.layer2.register_forward_hook(get_activation('feats'))
        elif 'layer3' in model_name:
            print("add hook in layer3")
            model.layer3.register_forward_hook(get_activation('feats'))
        elif 'layer4_no_pooling' in model_name:
            print("add hook in layer4_no_pooling")
            model.layer4.register_forward_hook(get_activation('feats'))
        else:
            model.fc.register_forward_hook(get_activation('feats'))
        
    # load data
    for exp_index, exp_name in enumerate(exp_names_list):
        print(model_name, exp_name)
        valdir = exp_name
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        _trans = transforms.Compose([
                transforms.Resize(224),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                normalize,
            ])

        ## read & sort img filenames
        all_filenames = []
        # read img filenames
        for filename in os.listdir(valdir):
            if filename[-4:]=='.png':
                # for basal dataset / new controled exps
                assert(filename[:13]=='canvasvisible' or filename[:8]=='trialnum')
                all_filenames.append(filename)
            
                
        ## sort img filenames -> fix bug of sorting by value
        # for basal dataset / new controled exps
        if all_filenames[0][:13]=='canvasvisible':
            all_filenames = sorted(all_filenames, key=lambda x: int(x[:-4].split("_")[-1][5:]))
        elif all_filenames[0][:8]=='trialnum':
            all_filenames = sorted(all_filenames, key=lambda x: int(x[8:-4]))
        else:
            assert False, 'unsupported filename'
        print(len(all_filenames))
        
        # extract feats
        FEATS = []
        LABELS = []
        # loop through batches
        for idx, filename in enumerate(all_filenames):
            # read & transform img
            img = Image.open(os.path.join(valdir, filename)).convert("RGB")
            img_trans = _trans(img)
            # move to device
            inputs = img_trans.unsqueeze(0).cuda()
            # forward pass [with feature extraction]
            preds = model(inputs)
            # preds = model.forward_single(inputs)
            # add feats and preds to lists
            # if 'simplecnn' in model_name:
            #     activation['feats'] = F.relu(activation['feats'])
            FEATS.append(activation['feats'].cpu().squeeze().unsqueeze(0).numpy())
            if save_label:
                LABELS.append(outputs.cpu().squeeze().unsqueeze(0).numpy())
        FEATS = np.concatenate(FEATS, axis=0)
        if save_label:
            LABELS = np.concatenate(LABELS, axis=0)
        print(FEATS.shape)
        
        if subsample:
            FEATS = FEATS.reshape([FEATS.shape[0], -1])
            if exp_index == 0: # use the same subset indexes for all exp
                # sampled_indexes = np.random.permutation()
                # sampled_indexes = np.random.permutation(FEATS.shape[1])[:2048]
                sampled_indexes = np.random.choice(FEATS.shape[1], 2048, replace=False)
            FEATS = FEATS[:, sampled_indexes]
            print(f"subsample to 2048 feats, {FEATS.shape}")
        
        # save feats
        filename = exp_name+'_'+model_name+filename_postfix+'_fc.pth'
        torch.save(FEATS, filename)
        print(f'saving to {filename}\n')
        
        if save_label:
            filename = exp_name+'_'+model_name+filename_postfix+'_label.pth'
            torch.save(LABELS, filename)


## extract pixels

In [ ]:
# (8) exp_names_list_GER_21_elias
useful_stats = {}

data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/'
]


exp_name_list = [
                ['vbsle151_elias_neutral', 'vbsle151_elias_happiness_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_sadness_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_disgust_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_sadness_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_disgust_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_disgust_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_disgust_4', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_disgust_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_disgust_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_fear_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_fear_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_anger_4', 'vbsle151_elias_surprise_4']

                ]

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10, 11, 12, 13,14, 15, 16, 17, 18, 19, 20]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
### extract pixels

# SL
# updated loc
exp_names_list = ['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_elias',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_neptune']

exp_names_list = [
                  # AppleFace Elias/Neptune 151 (w/ inverse contrast)
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_elias',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_neptune',

                  # AppleFace Elias/Neptune 101 
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_101/20210920_Var6vbsl_set0_im101_elias',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_101/20210920_Var6vbsl_set0_im101_neptune', 

                  # OOD AppleFace Sophie/Praneeth
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_sophie_praneeth/20230406_Var6vbsl_set0_im151_praneeth_dur200ms_lab',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_sophie_praneeth/20230406_Var6vbsl_set0_im151_sophie_dur200ms_lab',
                  
                  # OOD Basel (w/ texture)
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood/20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_dur200ms_lab',                  
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood/20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_dur200ms_lab',

                  # OOD Basel 
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_notexture_dur200ms_lab',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_notexture_dur200ms_lab',
                  
                  # OOD Object camel/elephant
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_camel',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant',
                  
                  # OOD Object camel/elephant
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20220412_Var6vbsp_set1_im101_camel',
                  '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20220412_Var6vb_same_as_camel_sp_set1_im101_elephant']

for exp_name in exp_names_list:
    valdir = exp_name

    ## read & sort img filenames
    all_filenames = []
    # read img filenames
    for filename in os.listdir(valdir):
        if filename[-4:]=='.png':
            # for basal dataset / new controled exps
            assert(filename[:13]=='canvasvisible' or filename[:8]=='trialnum')
            all_filenames.append(filename)

    ## sort img filenames -> fix bug of sorting by value
    # for basal dataset / new controled exps
    if all_filenames[0][:13]=='canvasvisible':
        all_filenames = sorted(all_filenames, key=lambda x: int(x[:-4].split("_")[-1][5:]))
    elif all_filenames[0][:8]=='trialnum':
        all_filenames = sorted(all_filenames, key=lambda x: int(x[8:-4]))
    else:
        assert False, 'unsupported filename'
    print(len(all_filenames))
    
    # extract feats
    FEATS = []
    # loop through batches
    for idx, filename in enumerate(all_filenames):
        # read & transform img
        img = Image.open(os.path.join(valdir, filename)).convert('L')
        img = img.resize((64, 64))
        FEATS.append(np.array(img).reshape(-1))
    FEATS = np.array(FEATS)
    print(FEATS.shape)

    # save feats
    filename = exp_name+'_'+'pixel'+'.pth'
    torch.save(FEATS, filename)
    print(f'saving to {filename}\n')

In [ ]:
model_names_list = ['lowpassfilter_colored_noblur', 
                    'lowpassfilter_colored_linearblur',
                    'lowpassfilter_colored_nonlinearblur',
                    # 'lowpassfilter_colored',
                    # 'lowpassfilter_colored_nonlinearblur1',
                    # 'lowpassfilter_colored_nonlinearblur2'
                    ]

In [ ]:
# (3) exp_names_list_GFR_21 - BASEL color texture

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_colored_texture/Basel_colored_texture_test_1000ID/'] * 66

exp_name_list = [
                ['id10', 'id11'],
                 ['id10', 'id12'],
                 ['id10', 'id13'],
                 ['id10', 'id14'],
                 ['id10', 'id15'],
                 ['id10', 'id16'],
                 ['id11', 'id12'],
                 ['id11', 'id13'],
                 ['id11', 'id14'],
                 ['id11', 'id15'],
                 ['id11', 'id16'],
                 ['id12', 'id13'],
                 ['id12', 'id14'],
                 ['id12', 'id15'],
                 ['id12', 'id16'],
                 ['id13', 'id14'],
                 ['id13', 'id15'],
                 ['id13', 'id16'],
                 ['id14', 'id15'],
                 ['id14', 'id16'],
                 ['id15', 'id16'],
                ['id10', 'id17'],
                ['id10', 'id18'],
                ['id10', 'id19'],
                ['id10', 'id20'],
                ['id10', 'id21'],
                ['id11', 'id17'],
                ['id11', 'id18'],
                ['id11', 'id19'],
                ['id11', 'id20'],
                ['id11', 'id21'],
                ['id12', 'id17'],
                ['id12', 'id18'],
                ['id12', 'id19'],
                ['id12', 'id20'],
                ['id12', 'id21'],
                ['id13', 'id17'],
                ['id13', 'id18'],
                ['id13', 'id19'],
                ['id13', 'id20'],
                ['id13', 'id21'],
                ['id14', 'id17'],
                ['id14', 'id18'],
                ['id14', 'id19'],
                ['id14', 'id20'],
                ['id14', 'id21'],
                ['id15', 'id17'],
                ['id15', 'id18'],
                ['id15', 'id19'],
                ['id15', 'id20'],
                ['id15', 'id21'],
                ['id16', 'id17'],
                ['id16', 'id18'],
                ['id16', 'id19'],
                ['id16', 'id20'],
                ['id16', 'id21'],
                ['id17', 'id18'],
                ['id17', 'id19'],
                ['id17', 'id20'],
                ['id17', 'id21'],
                ['id18', 'id19'],
                ['id18', 'id20'],
                ['id18', 'id21'],
                ['id19', 'id20'],
                ['id19', 'id21'],
                ['id20', 'id21'],
] 
   
    
# specify which dataset to use by index 
model_acc_across_tasks = {model_name: [] for model_name in model_names_list}


for dataset_index in range(52, 66):# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        model_acc_across_tasks[model_name].append(np.array(all_scores).mean())

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
for model_name, acc_list in model_acc_across_tasks.items():
    print(model_name)
    print('\t'.join(f"{acc}" for acc in acc_list))


# get bio i1s

## vbsli* i1

In [ ]:
# read file
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_neptune/human_n_subs=37_20211011_Var6vbsl_set0_im151_elias_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_neptune_dur200ms_lab_updated.json_I1_partition_splithalf_trial_binsize=35245_n_rep=5"
_path = "../data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_elias_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_neptune_dur200ms_lab_updated.json_n_rep=5"

# SL
_path = "/mnt/smb/locker/issa-locker/users/Josh/data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_elias_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_neptune_dur200ms_lab_updated.json_n_rep=5"

with open(_path, 'rb') as f:
    human_i1_5rep = pickle.load(f).squeeze()
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_neptune/AJ_176_20211011_Var6vbsl_set0_im151_neptune_dur400ms_lab_updated.json_20211011_Var6vbsl_set0_im151_elias_dur400ms_lab_updated.json_0_I1_partition_splithalf_trial_binsize=24008n_rep=5"
_path = "../data/face_data/bio_data/avg-monkey-i1_splithalf"

# SL
_path = "/mnt/smb/locker/issa-locker/users/Josh/data/face_data/bio_data/avg-monkey-i1_splithalf"

with open(_path, 'rb') as f:
    marmoset_i1_5rep = pickle.load(f).squeeze()
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_neptune/human_n_subs=37_20211011_Var6vbsl_set0_im151_elias_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_neptune_dur200ms_lab_updated.json_I1_partition_trial_binsize=35245"
_path = "../data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_elias_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_neptune_dur200ms_lab_updated.json"

# SL
_path = "/mnt/smb/locker/issa-locker/users/Josh/data/face_data/bio_data/human-avg-sub_20211011_Var6vbsl_set0_im151_elias_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_neptune_dur200ms_lab_updated.json"

with open(_path, 'rb') as f:
    human_i1 = pickle.load(f).squeeze().reshape(-1)
# _path = "../../data/face_data/20211011_Var6vbsl_set0_im151_neptune/AJ_176_20211011_Var6vbsl_set0_im151_neptune_dur400ms_lab_updated.json_20211011_Var6vbsl_set0_im151_elias_dur400ms_lab_updated.json_0_I1_partition_trial_binsize=24008"
_path = "../data/face_data/bio_data/avg-monkey-i1"

# SL
_path = "/mnt/smb/locker/issa-locker/users/Josh/data/face_data/bio_data/avg-monkey-i1"

with open(_path, 'rb') as f:
    marmoset_i1 = pickle.load(f).squeeze().reshape(-1)

In [ ]:
# normalize 
def NormalizeData(data, data_max=None, data_min=None): # normalize to [0,1]
    if data_max is None and data_min is None:
        return (data - np.min(data)) / (np.max(data) - np.min(data))
    else:
        return (data - data_min) / (data_max - data_min)
is_normalized = True
if is_normalized:
    human_i1_5rep = NormalizeData(human_i1_5rep, 4, -4)
    human_i1 = NormalizeData(human_i1, 4, -4)
    marmoset_i1_5rep = NormalizeData(marmoset_i1_5rep, 4, -4)
    marmoset_i1 = NormalizeData(marmoset_i1, 4, -4)
human_i1_5rep.shape, marmoset_i1_5rep.shape, human_i1.shape, marmoset_i1.shape

## human i1s - camel vs. elephant

In [ ]:
# read human i1s - camel vs. elephant
_path = "../../data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant/human_n_subs=31_20211011_Var6vbsl_set0_im151_camel_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_elephant_dur200ms_lab_updated.json_I1_partition_splithalf_trial_binsize=13738_n_rep=5"

# SL (path updated)
# Human experiment, using set0, im151 -- elephant, subs=31, 20211011
_path = "/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20211011_Var6vbsl_set0_im151_elephant/human_n_subs=31_20211011_Var6vbsl_set0_im151_camel_dur200ms_lab_updated.json_20211011_Var6vbsl_set0_im151_elephant_dur200ms_lab_updated.json_I1_partition_splithalf_trial_binsize=13738_n_rep=5"

with open(_path, 'rb') as f:
    human_i1s_obj_vbsl151 = pickle.load(f).squeeze()
_path = "../../data/face_data/control_exp/objs/20220412_Var6vb_same_as_camel_sp_set1_im101_elephant/human_n_subs=26_20220412_Var6vbsp_set1_im101_camel_dur200ms.json_20220412_Var6vb_same_as_camel_sp_set1_im101_elephant_dur200ms.json_I1_partition_splithalf_trial_binsize=10043_n_rep=5"

# SL (path updated)
# Human experiment, using set1, im101 -- elephant, subs=26, 20220412 
_path = "/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/20220412_Var6vb_same_as_camel_sp_set1_im101_elephant/human_n_subs=26_20220412_Var6vbsp_set1_im101_camel_dur200ms.json_20220412_Var6vb_same_as_camel_sp_set1_im101_elephant_dur200ms.json_I1_partition_splithalf_trial_binsize=10043_n_rep=5"

with open(_path, 'rb') as f:
    human_i1s_obj_vbsp101 = pickle.load(f).squeeze()

In [ ]:
# SL 
is_normalized = True
if is_normalized:
    human_i1s_obj_vbsl151 = NormalizeData(human_i1s_obj_vbsl151)
    human_i1s_obj_vbsp101 = NormalizeData(human_i1s_obj_vbsp101)
human_i1s_obj_vbsl151.shape, human_i1s_obj_vbsp101.shape

In [ ]:
human_i1s_obj_vbsl151.shape, human_i1s_obj_vbsp101.shape

# compute model i1s

In [ ]:

src = 'barlowtwins_finetune_7way_EM_seojin_seed77__model_best.pth.tar'
dst  = '/mnt/smb/locker/issa-locker/users/Seojin/saved_models/barlowtwins_finetune_7way_EM_seojin_seed77__model_best.pth.tar'
shutil.copyfile(src, dst)

print(f"Copied file to: {dst}")

In [ ]:
# (3) exp_names_list_GFR_21

useful_stats = {}
data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/'
]


exp_name_list = [
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_neptune/vbsle151_neptune_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_seojin/vbsle151_seojin_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_sophie/vbsle151_sophie_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_seojin/vbsle151_seojin_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_sophie/vbsle151_sophie_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_sophie/vbsle151_sophie_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_younah/vbsle151_younah_neutral']
]
    
# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18, 19,20]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
model_names_list=[
# 'off_the_shelf_barlowtwins_texture_colorbg_2way_seojin_dan',
# 'off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_seojin',
# 'off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_kedar',
# 'off_the_shelf_barlowtwins_finetune_32way_8ID_4EM_IDEM_colorbg',
# 'SL_resnet50_finetune_vbsle_50k_elias_neptune_seed777_model_best',
# 'Josh_256bs_pretrained_model_best',
# 'off_the_shelf_barlowtwins_IDEM_28way_colorbg_epochs50'
'lowpassfilter_grayscale_linearblur'
]

In [ ]:
# (4) exp_names_list_GFR_additional
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/'] * 45
useful_stats = {}
exp_name_list = [
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_younah/vbsle151_younah_neutral', 'vbsle151_ashley/vbsle151_ashley_neutral'],
                ['vbsle151_younah/vbsle151_younah_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_younah/vbsle151_younah_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_younah/vbsle151_younah_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_younah/vbsle151_younah_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_ashley/vbsle151_ashley_neutral', 'vbsle151_kedar/vbsle151_kedar_neutral'],
                ['vbsle151_ashley/vbsle151_ashley_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_ashley/vbsle151_ashley_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_ashley/vbsle151_ashley_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_kedar/vbsle151_kedar_neutral', 'vbsle151_josh/vbsle151_josh_neutral'],
                ['vbsle151_kedar/vbsle151_kedar_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_kedar/vbsle151_kedar_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_josh/vbsle151_josh_neutral', 'vbsle151_ani/vbsle151_ani_neutral'],
                ['vbsle151_josh/vbsle151_josh_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
                ['vbsle151_ani/vbsle151_ani_neutral', 'vbsle151_tony/vbsle151_tony_neutral'],
] 

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40, 41, 42, 43, 44]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists




In [ ]:
# Print the shape and a few sample feature values
print("Extracted feature shape:", FEATS.shape)
print("Sample features for the first image pair:")
print(FEATS[0])  # Print the first feature vector


In [ ]:
# (5) exp_names_list_GFR_21_texture_colorbg

useful_stats = {}
data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'
]


exp_name_list = [
                ['vbsl151_elias_neutral', 'vbsl151_neptune_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_seojin_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_sophie_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_seojin_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_sophie_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_sophie_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_sophie_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_sophie_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_sophie_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_dan_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_dan_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_sreyas_neutral', 'vbsl151_younah_neutral']
]

    
# specify which dataset to use by index 
for dataset_index in [0, 1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18, 19,20]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (9) exp_names_list_GER_21_elias_texture_colorbg
# model_names_list = ['SL_resnet50_finetune_vbsle_50k_10way_seed77_model_best', 'SL_resnet50_finetune_vbsle_50k_12way_seed77_model_best_12way']
useful_stats = {}

data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'
]

exp_name_list = [
                ['vbsl151_elias_neutral', 'vbsl151_elias_happiness_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_sadness_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_disgust_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_sadness_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_disgust_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_disgust_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_disgust_4', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_disgust_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_disgust_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_fear_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_fear_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_anger_4', 'vbsl151_elias_surprise_4']
]

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (11) exp_names_list_GFR_additional_texture_colorbg
useful_stats = {}
# (4) exp_names_list_GFR_additional
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'] * 45

exp_name_list = [
                ['vbsl151_elias_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_elias_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_elias_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_elias_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_elias_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_neptune_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_neptune_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_neptune_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_neptune_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_neptune_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_seojin_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_seojin_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_seojin_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_seojin_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_seojin_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_sophie_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_sophie_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_sophie_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_sophie_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_sophie_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_dan_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_dan_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_dan_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_dan_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_dan_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_sreyas_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_sreyas_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_sreyas_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_sreyas_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_sreyas_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_younah_neutral', 'vbsl151_ashley_neutral'],
                ['vbsl151_younah_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_younah_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_younah_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_younah_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_ashley_neutral', 'vbsl151_kedar_neutral'],
                ['vbsl151_ashley_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_ashley_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_ashley_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_kedar_neutral', 'vbsl151_josh_neutral'],
                ['vbsl151_kedar_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_kedar_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_josh_neutral', 'vbsl151_ani_neutral'],
                ['vbsl151_josh_neutral', 'vbsl151_tony_neutral'],
                ['vbsl151_ani_neutral', 'vbsl151_tony_neutral'],
] 

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10, 11, 12, 13,14, 15, 16, 17, 18, 19, 20, 21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40, 41, 42, 43, 44]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists




In [ ]:
# (8) exp_names_list_GER_21_elias
useful_stats = {}

data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/vbsle151_elias/'
]


exp_name_list = [
                ['vbsle151_elias_neutral', 'vbsle151_elias_happiness_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_sadness_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_disgust_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_neutral', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_sadness_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_disgust_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_happiness_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_disgust_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_sadness_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_disgust_4', 'vbsle151_elias_fear_4'],
                 ['vbsle151_elias_disgust_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_disgust_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_fear_4', 'vbsle151_elias_anger_4'],
                 ['vbsle151_elias_fear_4', 'vbsle151_elias_surprise_4'],
                 ['vbsle151_elias_anger_4', 'vbsle151_elias_surprise_4']

                ]

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10, 11, 12, 13,14, 15, 16, 17, 18, 19, 20]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (9) exp_names_list_GER_21_elias_texture_colorbg

useful_stats = {}

data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'
]

exp_name_list = [
                ['vbsl151_elias_neutral', 'vbsl151_elias_happiness_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_sadness_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_disgust_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_neutral', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_sadness_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_disgust_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_happiness_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_happiness_4',
                 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_disgust_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_sadness_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_disgust_4', 'vbsl151_elias_fear_4'],
                ['vbsl151_elias_disgust_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_disgust_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_fear_4', 'vbsl151_elias_anger_4'],
                ['vbsl151_elias_fear_4', 'vbsl151_elias_surprise_4'],
                ['vbsl151_elias_anger_4', 'vbsl151_elias_surprise_4']
]

# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (3) exp_names_list_GFR_21 - BASEL

useful_stats = {}
data_root_list = ['/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/Basel_no_texture/Basel_no_texture_test_1000ID/'] * 66

exp_name_list = [
                ['id988', 'id989'],
                 ['id988', 'id990'],
                 ['id988', 'id991'],
                 ['id988', 'id992'],
                 ['id988', 'id993'],
                 ['id988', 'id994'],
                 ['id989', 'id990'],
                 ['id989', 'id991'],
                 ['id989', 'id992'],
                 ['id989', 'id993'],
                 ['id989', 'id994'],
                 ['id990', 'id991'],
                 ['id990', 'id992'],
                 ['id990', 'id993'],
                 ['id990', 'id994'],
                 ['id991', 'id992'],
                 ['id991', 'id993'],
                 ['id991', 'id994'],
                 ['id992', 'id993'],
                 ['id992', 'id994'],
                 ['id993', 'id994'],
                ['id988', 'id995'],
                ['id988', 'id996'],
                ['id988', 'id997'],
                ['id988', 'id998'],
                ['id988', 'id999'],
                ['id989', 'id995'],
                ['id989', 'id996'],
                ['id989', 'id997'],
                ['id989', 'id998'],
                ['id989', 'id999'],
                ['id990', 'id995'],
                ['id990', 'id996'],
                ['id990', 'id997'],
                ['id990', 'id998'],
                ['id990', 'id999'],
                ['id991', 'id995'],
                ['id991', 'id996'],
                ['id991', 'id997'],
                ['id991', 'id998'],
                ['id991', 'id999'],
                ['id992', 'id995'],
                ['id992', 'id996'],
                ['id992', 'id997'],
                ['id992', 'id998'],
                ['id992', 'id999'],
                ['id993', 'id995'],
                ['id993', 'id996'],
                ['id993', 'id997'],
                ['id993', 'id998'],
                ['id993', 'id999'],
                ['id994', 'id995'],
                ['id994', 'id996'],
                ['id994', 'id997'],
                ['id994', 'id998'],
                ['id994', 'id999'],
                ['id995', 'id996'],
                ['id995', 'id997'],
                ['id995', 'id998'],
                ['id995', 'id999'],
                ['id996', 'id997'],
                ['id996', 'id998'],
                ['id996', 'id999'],
                ['id997', 'id998'],
                ['id997', 'id999'],
                ['id998', 'id999'],
] 
   
model_acc_across_tasks = {model_name: [] for model_name in model_names_list}

# specify which dataset to use by index 
for dataset_index in range(66):# for dataset_index in [3,4,5]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        model_acc_across_tasks[model_name].append(np.array(all_scores).mean())

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

print("\n=== Summary: Average Accuracy Across Datasets ===")
for model_name in model_names_list:
    accs = []
    for data_root in data_root_list:
        if model_name in useful_stats[data_root]:
            acc_mean, acc_std = useful_stats[data_root][model_name]['acc']
            accs.append(acc_mean)
    if accs:
        overall_mean = np.mean(accs)
        overall_std = np.std(accs)
        print(f"{model_name}: {overall_mean:.4f} ± {overall_std:.4f}")




In [ ]:
# import pickle
# useful_stats = {}
# with open('useful_stats_ablation.pkl', 'wb') as f:SL_20240715
#     pickle.dump(useful_stats, f)

# useful_stats = {}

import pickle
# SL Path updated
with open('useful_stats_SL_20240715.pkl', 'rb') as f: # useful_stats_ablation, useful_stats_tuning, useful_stats_penn
    useful_stats = pickle.load(f)
    
# hierachy: [task_dir][model_name][acc/acc_per_img/dist/i1]
# acc [list]: acc over all datapoints and trails -> [acc_mean, acc_std]
# acc_per_img [array]: acc per image averaged over all trails -> shape [2, num_img_per_class]
# dist [array]: all svm distance (note that the second class dist is negated) -> shape [2, num_repetition, num_img_per_class]
# i1_5rep [array]: random split-half i1 with 5 repetition -> shape [5, 2*num_img_per_class]
# i1 [array]: model i1 (the same as dist averaging over trails) -> shape [2, num_img_per_class]
# i1_corr [list]: model i1 correlation with bio systems. For vbsl101, the order is [Bourgeois, Sausage, AJ]. 
# For vbsl151, the order is [Human, AJ]. For each corr, I record [i1_corr_mean, i1_corr_std]

In [ ]:
for model_name, acc_list in model_acc_across_tasks.items():
    print(model_name)
    print('\t'.join(f"{acc}" for acc in acc_list))


In [ ]:
import os # START HERE
import numpy as np
from PIL import Image
import torch
from torchvision import transforms
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from load_model import load_model
# 4200 images, category order

combined_folder = '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsl50_texture/combined_vbsl50_texture_registered' # contains 4200 images
combined_folder = '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsl50_registered/combined_vbsl50_registered' # contains 4200 images

save_rdm_path = '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/correlation_analysis/vbsl50_registered_avgpool/'

os.makedirs(save_rdm_path, exist_ok=True)

# Strict category order (identity_emotion)
category_order = [
    "Elias_neutral", "Elias_happiness_4", "Elias_sadness_4", "Elias_surprise_4", "Elias_fear_4", "Elias_disgust_4", "Elias_anger_4",
    "Neptune_neutral", "Neptune_happiness_4", "Neptune_sadness_4", "Neptune_surprise_4", "Neptune_fear_4", "Neptune_disgust_4", "Neptune_anger_4",
    "Seojin_neutral", "Seojin_happiness_4", "Seojin_sadness_4", "Seojin_surprise_4", "Seojin_fear_4", "Seojin_disgust_4", "Seojin_anger_4",
    "Sophie_neutral", "Sophie_happiness_4", "Sophie_sadness_4", "Sophie_surprise_4", "Sophie_fear_4", "Sophie_disgust_4", "Sophie_anger_4",
    "Dan_neutral", "Dan_happiness_4", "Dan_sadness_4", "Dan_surprise_4", "Dan_fear_4", "Dan_disgust_4", "Dan_anger_4",
    "Sreyas_neutral", "Sreyas_happiness_4", "Sreyas_sadness_4", "Sreyas_surprise_4", "Sreyas_fear_4", "Sreyas_disgust_4", "Sreyas_anger_4",
    "Younah_neutral", "Younah_happiness_4", "Younah_sadness_4", "Younah_surprise_4", "Younah_fear_4", "Younah_disgust_4", "Younah_anger_4",
    "Ashley_neutral", "Ashley_happiness_4", "Ashley_sadness_4", "Ashley_surprise_4", "Ashley_fear_4", "Ashley_disgust_4", "Ashley_anger_4",
    "Josh_neutral", "Josh_happiness_4", "Josh_sadness_4", "Josh_surprise_4", "Josh_fear_4", "Josh_disgust_4", "Josh_anger_4",
    "Kedar_neutral", "Kedar_happiness_4", "Kedar_sadness_4", "Kedar_surprise_4", "Kedar_fear_4", "Kedar_disgust_4", "Kedar_anger_4",
    "Ani_neutral", "Ani_happiness_4", "Ani_sadness_4", "Ani_surprise_4", "Ani_fear_4", "Ani_disgust_4", "Ani_anger_4",
    "Tony_neutral", "Tony_happiness_4", "Tony_sadness_4", "Tony_surprise_4", "Tony_fear_4", "Tony_disgust_4", "Tony_anger_4"
]


# model_names_list = ['resnet50']
# category_order = [
#     "Elias_neutral_textured", "Elias_happiness_4_textured", "Elias_sadness_4_textured", "Elias_surprise_4_textured", "Elias_fear_4_textured", "Elias_disgust_4_textured", "Elias_anger_4_textured",
#     "Neptune_neutral_textured", "Neptune_happiness_4_textured", "Neptune_sadness_4_textured", "Neptune_surprise_4_textured", "Neptune_fear_4_textured", "Neptune_disgust_4_textured", "Neptune_anger_4_textured",
#     "Seojin_neutral_textured", "Seojin_happiness_4_textured", "Seojin_sadness_4_textured", "Seojin_surprise_4_textured", "Seojin_fear_4_textured", "Seojin_disgust_4_textured", "Seojin_anger_4_textured",
#     "Sophie_neutral_textured", "Sophie_happiness_4_textured", "Sophie_sadness_4_textured", "Sophie_surprise_4_textured", "Sophie_fear_4_textured", "Sophie_disgust_4_textured", "Sophie_anger_4_textured",
#     "Dan_neutral_textured", "Dan_happiness_4_textured", "Dan_sadness_4_textured", "Dan_surprise_4_textured", "Dan_fear_4_textured", "Dan_disgust_4_textured", "Dan_anger_4_textured",
#     "Sreyas_neutral_textured", "Sreyas_happiness_4_textured", "Sreyas_sadness_4_textured", "Sreyas_surprise_4_textured", "Sreyas_fear_4_textured", "Sreyas_disgust_4_textured", "Sreyas_anger_4_textured",
#     "Younah_neutral_textured", "Younah_happiness_4_textured", "Younah_sadness_4_textured", "Younah_surprise_4_textured", "Younah_fear_4_textured", "Younah_disgust_4_textured", "Younah_anger_4_textured",
#     "Ashley_neutral_textured", "Ashley_happiness_4_textured", "Ashley_sadness_4_textured", "Ashley_surprise_4_textured", "Ashley_fear_4_textured", "Ashley_disgust_4_textured", "Ashley_anger_4_textured",
#     "Josh_neutral_textured", "Josh_happiness_4_textured", "Josh_sadness_4_textured", "Josh_surprise_4_textured", "Josh_fear_4_textured", "Josh_disgust_4_textured", "Josh_anger_4_textured",
#     "Kedar_neutral_textured", "Kedar_happiness_4_textured", "Kedar_sadness_4_textured", "Kedar_surprise_4_textured", "Kedar_fear_4_textured", "Kedar_disgust_4_textured", "Kedar_anger_4_textured",
#     "Ani_neutral_textured", "Ani_happiness_4_textured", "Ani_sadness_4_textured", "Ani_surprise_4_textured", "Ani_fear_4_textured", "Ani_disgust_4_textured", "Ani_anger_4_textured",
#     "Tony_neutral_textured", "Tony_happiness_4_textured", "Tony_sadness_4_textured", "Tony_surprise_4_textured", "Tony_fear_4_textured", "Tony_disgust_4_textured", "Tony_anger_4_textured"
# ]

# model_names_list = [
#     'SL_resnet50_finetune_Basel_color_texture_2id_seed777_model_best',
#     'SL_resnet50_finetune_Basel_color_texture_6id_seed77_model_best',
#     'SL_resnet50_finetune_Basel_color_texture_10id_seed77_model_best',
#     'SL_resnet50_finetune_Basel_color_texture_20ID_seed777',
#     'SL_resnet50_finetune_Basel_color_texture_30ID_seed777',
#     'SL_resnet50_finetune_Basel_color_texture_40ID_seed777',
#     'SL_resnet50_finetune_Basel_color_texture_50ID_seed777',
#     'SL_resnet50_finetune_Basel_color_texture_100id_seed777_model_best',
#     'SL_resnet50_finetune_Basel_color_texture_500id_seed777__best',
#     'SL_resnet50_finetune_Basel_color_texture_1kid_seed777_model_best',
# 'SL_resnet50_finetune_Basel_no_texture_2ID_seed777_model_best_SL',
# 'SL_resnet50_finetune_Basel_no_texture_4ID_seed777_model_best_SL',
# 'SL_resnet50_finetune_Basel_no_texture_6ID_seed777_model_best_SL',
# 'SL_resnet50_finetune_Basel_no_texture_8ID_seed777_model_best_SL',
# 'SL_resnet50_finetune_Basel_no_texture_10ID_seed777_model_best_SL',
# 'SL_resnet50_finetune_Basel_no_texture_12ID_seed77_model_best',
# 'SL_resnet50_finetune_Basel_no_texture_20ID_seed77_model_best',
# 'SL_resnet50_finetune_Basel_no_texture_30ID_seed777_',
# 'SL_resnet50_finetune_Basel_no_texture_40ID_seed777_',
# 'SL_resnet50_finetune_Basel_no_texture_50ID_seed777_',
# 'SL_resnet50_finetune_Basel_no_texture_500ID2_seed77_model_best',
# 'SL_resnet50_finetune_Basel_no_texture_100ID2_seed777_model_best',
# 'SL_resnet50_finetune_Basel_no_texture_1000ID_seed777_model_best'
# ]
# Image preprocessing
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])
# ==== Main Loop ====
for j, model_name in enumerate(model_names_list):
    print(f"Processing model: {model_name}")
    
    model = load_model(model_name)
    model.eval()

    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    if 'simplecnn' in model_name:
        model.fc2.register_forward_hook(get_activation('feats'))
    elif 'alexnet' in model_name:
        if 'layer4' in model_name:
            print("add hook in layer4")
            model.features[4].register_forward_hook(get_activation('feats'))
        elif 'layer8' in model_name:
            print("add hook in layer8")
            model.features[8].register_forward_hook(get_activation('feats'))
        elif 'layer12' in model_name:
            print("add hook in layer12")
            model.features[12].register_forward_hook(get_activation('feats'))
        elif 'layerC2' in model_name:
            model.classifier[2].register_forward_hook(get_activation('feats'))
        else: # final layer
            model.classifier[5].register_forward_hook(get_activation('feats'))
    elif model_name == 'vgg16':
        model.classifier[4].register_forward_hook(get_activation('feats'))
    elif model_name == 'vggface':
        model.fc7.register_forward_hook(get_activation('feats'))
    elif 'onlyDM' in model_name:
        model.depth_predictor.encoder[4].register_forward_hook(get_activation('feats'))
        # model.depth_predictor.decoder[30].register_forward_hook(get_activation('feats'))
        # model.depth_predictor.encoder[6].register_forward_hook(get_activation('feats'))
    else: # resnet50
        if 'layer1' in model_name:
            print("add hook in layer1")
            model.layer1.register_forward_hook(get_activation('feats'))
        elif 'layer2' in model_name:
            print("add hook in layer2")
            model.layer2.register_forward_hook(get_activation('feats'))
        elif 'layer3' in model_name:
            print("add hook in layer3")
            model.layer3.register_forward_hook(get_activation('feats'))
        elif 'layer4_no_pooling' in model_name:
            print("add hook in layer4_no_pooling")
            model.layer4.register_forward_hook(get_activation('feats'))
        else:
            model.avgpool.register_forward_hook(get_activation('feats'))
        

    # Step 1: Extract all 4200 features in strict order
    all_feats = []

    for category in category_order:
        for i in range(50):
            fname = f"{category}_trialnum{i}.png"
            fpath = os.path.join(combined_folder, fname)
            if not os.path.exists(fpath):
                print(f"Missing: {fpath}")
                continue
            img = Image.open(fpath).convert("RGB")
            img_tensor = transform(img).unsqueeze(0).cuda()
            with torch.no_grad():
                _ = model(img_tensor)
            feat = activation['feats'].cpu().squeeze().numpy().flatten()
            all_feats.append(feat)

    all_feats = np.stack(all_feats).astype(np.float32)  # shape: (4200, D)

    # Step 2: Compute full 4200x4200 correlation RDM
    full_rdm = 1 - np.corrcoef(all_feats)  # shape: (4200, 4200)

    # Step 3: Block-average into 84x84
    block_size = 50
    num_categories = len(category_order)
    avg_rdm = np.zeros((num_categories, num_categories))

    for i in range(num_categories):
        for j in range(num_categories):
            block = full_rdm[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            avg_rdm[i, j] = block.mean()

    # Step 4: Save
    rdm_path = os.path.join(save_rdm_path, f"combined_vbsl50_{model_name}_rdm_blockavg.npy")
    np.save(rdm_path, avg_rdm)
    print(f"Saved block-averaged RDM to: {rdm_path}")

In [ ]:

model_names_list = [
#  'SL_resnet50_finetune_vbsle_50k_texture_colorbg_seojin_ani_seed777_model_best',
#     'SL_resnet50_finetune_vbsle_50k_texture_colorbg_seojin_tony_seed777_model_best',
#     'SL_resnet50_finetune_vbsle_50k_texture_colorbg_seojin_kedar_seed777_model_best',
    'off_the_shelf_barlowtwins_finetune_12way_6ID_2EM_IDEM_colorbg',
    'off_the_shelf_barlowtwins_finetune_24way_6ID_4EM_IDEM_colorbg',
    'off_the_shelf_barlowtwins_finetune_16way_8ID_2EM_IDEM_colorbg',
    'off_the_shelf_barlowtwins_finetune_32way_8ID_4EM_IDEM_colorbg',
]

In [ ]:
from load_model_ver3 import load_model
model_names_list = [
 "SL_resnet50_finetune_vbsle_50k_4way_SSKD_seed77_model_best",
"SL_resnet50_finetune_vbsle_50k_6way_final_seed77_model_best",
"SL_resnet50_finetune_vbsle_50k_8way_far_seed77_model_best",
"SL_resnet50_finetune_7way_EM_seojin_seed77_model_best",
"SL_resnet50_finetune_vbsle_50k_sophie_seojin_colorbg_seed777_model_best",
"SL_resnet50_finetune_texture_colorbg_2way_seojin_dan_seed777_model_best",
"SL_resnet50_finetune_vbsle_50k_sophie_kedar_colorbg_seed777_model_best",
"SL_resnet50_finetune_vbsle_50k_4way_SSKD_colorbg_seed77_model_best",
"SL_resnet50_finetune_vbsle_50k_6way_final_colorbg_seed77_model_best",
"SL_resnet50_finetune_texture_colorbg_8way_seed77_model_best",
"SL_resnet50_finetune_texture_colorbg_em_neutral_anger_seed777_model_best",
"SL_resnet50_finetune_texture_colorbg_em_4way_NHAS_seed77_model_best",
"SL_colorbg_resnet50_finetune_7way_EM_seojin_seed77_model_best",
"SL_resnet50_finetune_7way_EM_sophie_colorbg_seed77_model_best",
"SL_resnet50_finetune_7way_EM_dan_colorbg_seed77__model_best",
"SL_resnet50_finetune_7way_EM_kedar_colorbg_seed77__model_best",
"SL_resnet50_finetune_4way_IDEM_seojin_sophie_colorbg_seed77_model_best",
"SL_resnet50_finetune_4way_IDEM_seojin_dan_colorbg_seed77_model_best",
"SL_resnet50_finetune_4way_IDEM_sophie_kedar_colorbg_seed77_model_best",
"SL_resnet50_finetune_8way_IDEM_seojin_sophie_colorbg_seed77_model_best",
"SL_resnet50_finetune_8way_IDEM_seojin_dan_colorbg_seed77_model_best",
"SL_resnet50_finetune_8way_IDEM_sophie_kedar_colorbg_seed77_model_best",
"SL_resnet50_finetune_14way_IDEM_seojin_sophie_colorbg_seed77_model_best",
"SL_resnet50_finetune_14way_IDEM_seojin_dan_colorbg_seed777_model_best",
"SL_resnet50_finetune_14way_IDEM_sophie_kedar_colorbg_seed777_model_best",
"SL_resnet50_finetune_8way_IDEM_ssskd_colorbg_seed77_model_best",
"SL_resnet50_finetune_16way_IDEM_ssskd_colorbg_seed777",
"SL_resnet50_finetune_28way_IDEM_colorbg_seed777_model_best",
"SL_resnet50_finetune_12way_6ID_2EM_IDEM_colorbg",
"SL_resnet50_finetune_24way_6ID_4EM_IDEM_colorbg",
"SL_resnet50_finetune_42way_IDEM_colorbg_seed777_model_best",
"SL_resnet50_finetune_16way_8ID_2EM_IDEM_colorbg",
"SL_resnet50_finetune_32way_8ID_4EM_IDEM_colorbg",
"SL_resnet50_finetune_56way_IDEM_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_vbsle_50k_4way_SSKD_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_vbsle_50k_6way_final_one",
"off_the_shelf_barlowtwins_ID_8way_epochs50",
"off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_seojin",
"off_the_shelf_barlowtwins_texture_colorbg_2way_seojin_dan",
"off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_kedar",
"off_the_shelf_barlowtwins_finetune_vbsle_50k_4way_SSKD_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_6way_final_colorbg",
"off_the_shelf_barlowtwins_ID_8way_colorbg_epochs50",
"off_the_shelf_barlowtwins_finetune_texture_colorbg_em_neutral_anger",
"off_the_shelf_barlowtwins_finetune_texture_colorbg_em_4way_NHAS",
"off_the_shelf_barlowtwins_finetune_7way_EM_seojin_seed77_texture_model_best",
"off_the_shelf_barlowtwins_finetune_7way_EM_sophie_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_7way_EM_dan_colorbg_seed77__model_best",
"off_the_shelf_barlowtwins_finetune_7way_EM_kedar_colorbg_seed77__model_best",
"off_the_shelf_barlowtwins_finetune_4way_IDEM_seojin_sophie_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_4way_IDEM_seojin_dan_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_4way_IDEM_sophie_kedar_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_8way_IDEM_seojin_sophie_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_8way_IDEM_seojin_dan_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_finetune_8way_IDEM_sophie_kedar_colorbg_seed77_model_best",
"off_the_shelf_barlowtwins_IDEM_14way_colorbg",
"off_the_shelf_barlowtwins_finetune_14way_IDEM_seojin_dan_colorbg_seed777_model_best",
"off_the_shelf_barlowtwins_finetune_14way_IDEM_sophie_kedar_colorbg_seed777_model_best",
"off_the_shelf_barlowtwins_finetune_8way_IDEM_ssskd_colorbg",
"off_the_shelf_barlowtwins_finetune_16way_IDEM_ssskd_colorbg",
"off_the_shelf_barlowtwins_IDEM_28way_colorbg_epochs50",
"off_the_shelf_barlowtwins_finetune_12way_6ID_2EM_IDEM_colorbg",
"off_the_shelf_barlowtwins_finetune_24way_6ID_4EM_IDEM_colorbg",
"off_the_shelf_barlowtwins_finetune_42way_IDEM_colorbg_seed777_model_best",
"off_the_shelf_barlowtwins_finetune_16way_8ID_2EM_IDEM_colorbg",
"off_the_shelf_barlowtwins_finetune_32way_8ID_4EM_IDEM_colorbg",
"off_the_shelf_barlowtwins_finetune_56way_IDEM_colorbg_seed77_model_best"
]
for model_name in model_names_list :
    model = load_model(model_name)

In [ ]:
model_names_list = [
    # 'SL_resnet50_finetune_7way_EM_seojin_seed77_model_best',
# 'SL_resnet50_finetune_7way_EM_sophie_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_7way_EM_dan_colorbg_seed77__model_best',
# 'SL_resnet50_finetune_7way_EM_kedar_colorbg_seed77__model_best',

# 'SL_resnet50_finetune_4way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_4way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_4way_IDEM_sophie_kedar_colorbg_seed77_model_best',

# 'SL_resnet50_finetune_6way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_6way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_6way_IDEM_sophie_kedar_colorbg_seed77_model_best',

# 'SL_resnet50_finetune_8way_IDEM_seojin_sophie_colorbg_seed77_model_best', 
# 'SL_resnet50_finetune_8way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_8way_IDEM_sophie_kedar_colorbg_seed77_model_best',

# 'SL_resnet50_finetune_10way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_10way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_10way_IDEM_sophie_kedar_colorbg_seed77_model_best',

# 'SL_resnet50_finetune_12way_IDEM_seojin_sophie_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_12way_IDEM_seojin_dan_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_12way_IDEM_sophie_kedar_colorbg_seed77_model_best',

#         "SL_resnet50_finetune_14way_IDEM_seojin_sophie_colorbg_seed77_model_best",
#         'SL_resnet50_finetune_14way_IDEM_seojin_dan_colorbg_seed777_model_best',
#         'SL_resnet50_finetune_14way_IDEM_sophie_kedar_colorbg_seed777_model_best',

# 'SL_resnet50_finetune_28way_IDEM_colorbg_seed777_model_best',
# 'SL_resnet50_finetune_42way_IDEM_colorbg_seed777_model_best',
# 'SL_resnet50_finetune_56way_IDEM_colorbg_seed77_model_best',
# 'SL_resnet50_finetune_12way_6ID_2EM_IDEM_colorbg',
# 'SL_resnet50_finetune_24way_6ID_4EM_IDEM_colorbg',
# 'SL_resnet50_finetune_16way_8ID_2EM_IDEM_colorbg',
# 'SL_resnet50_finetune_32way_8ID_4EM_IDEM_colorbg',


#     'off_the_shelf_barlowtwins_seojin_dan_epochs50',
#     'off_the_shelf_barlowtwins_2way_seojin_sophie',
#     'off_the_shelf_barlowtwins_2way_sophie_kedar',
#     'off_the_shelf_barlowtwins_finetune_4way_SSKD',
#     'off_the_shelf_barlowtwins_finetune_vbsle_50k_6way_final_one',
#     'off_the_shelf_barlowtwins_ID_8way_epochs50',

#     'off_the_shelf_barlowtwins_texture_colorbg_2way_seojin_dan',
#     'off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_seojin', 
#     'off_the_shelf_barlowtwins_texture_colorbg_2way_sophie_kedar',
#     'off_the_shelf_barlowtwins_texture_colorbg_4way',
#     'off_the_shelf_barlowtwins_finetune_6way_final_colorbg',
#     'off_the_shelf_barlowtwins_ID_8way_colorbg_epochs50',

#     'off_the_shelf_barlowtwins_finetune_texture_colorbg_em_neutral_anger',
#     'off_the_shelf_barlowtwins_finetune_texture_colorbg_em_4way_NHAS',
#     'off_the_shelf_barlowtwins_finetune_texture_colorbg_em_6way_excl_fear',

# 'off_the_shelf_barlowtwins_finetune_7way_EM_seojin_seed77_texture_model_best',
# 'off_the_shelf_barlowtwins_finetune_7way_EM_sophie_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_7way_EM_dan_colorbg_seed77__model_best',
'off_the_shelf_barlowtwins_finetune_7way_EM_kedar_colorbg_seed77__model_best',

'off_the_shelf_barlowtwins_finetune_4way_IDEM_seojin_sophie_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_4way_IDEM_seojin_dan_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_4way_IDEM_sophie_kedar_colorbg_seed77_model_best',

'off_the_shelf_barlowtwins_finetune_6way_IDEM_seojin_sophie_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_6way_IDEM_seojin_dan_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_6way_IDEM_sophie_kedar_colorbg_seed77_model_best',

'off_the_shelf_barlowtwins_finetune_8way_IDEM_seojin_sophie_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_8way_IDEM_seojin_dan_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_8way_IDEM_sophie_kedar_colorbg_seed77_model_best',

'off_the_shelf_barlowtwins_finetune_10way_IDEM_seojin_sophie_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_10way_IDEM_seojin_dan_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_10way_IDEM_sophie_kedar_colorbg_seed77_model_best',

'off_the_shelf_barlowtwins_finetune_12way_IDEM_seojin_sophie_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_12way_IDEM_seojin_dan_colorbg_seed77_model_best',
'off_the_shelf_barlowtwins_finetune_12way_IDEM_sophie_kedar_colorbg_seed77_model_best',

'off_the_shelf_barlowtwins_IDEM_28way_colorbg_epochs50',
'off_the_shelf_barlowtwins_finetune_42way_IDEM_colorbg_seed777_model_best',
    'off_the_shelf_barlowtwins_finetune_56way_IDEM_colorbg_seed77_model_best'
]

In [ ]:
dataset_name_list = ['face-vbsl151', 'face-vbsl101', 'face-ood-vbsl151', 
                    'basel-ood', 'basel-ood-notexture',
#                    'basel-ood-vbsl101', 'basel-ood-notexture-vbsl101', 
                    'obj-vbsl151', ]

data_root_list = ['../../data/face_data/vbsl_151/',
                  '../../data/face_data/control_exp/objs/',
                 '../../data/face_data/control_exp/ood/vbsl151_sophie_praneeth/',
                 '../../data/face_data/control_exp/ood/vbsl151_basel_ood/',
                 '../../data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/',
                 '../../data/face_data/control_exp/ood/vbsl101_ood_texture/',
                 '../../data/face_data/control_exp/ood/vbsl101_ood_notexture/', 
                 '../../data/face_data/vbsl_101/',]

# SL (update path)
data_root_list = ['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/',
                 '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_101/',
                 '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_sophie_praneeth/',
                 '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood/',
                 '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/',
                 '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/', 
                ]

useful_stats = {}

exp_name_list = [['20211011_Var6vbsl_set0_im151_elias', '20211011_Var6vbsl_set0_im151_neptune'],
                    ['20210920_Var6vbsl_set0_im101_elias', '20210920_Var6vbsl_set0_im101_neptune'],
                    ['20230406_Var6vbsl_set0_im151_praneeth_dur200ms_lab', '20230406_Var6vbsl_set0_im151_sophie_dur200ms_lab'],
                    ['20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_dur200ms_lab', '20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_dur200ms_lab'],
                    ['20230406_Var6vbsl_set0_im151_Baselmesh_ood_0_notexture_dur200ms_lab', '20230406_Var6vbsl_set0_im151_Baselmesh_ood_1_notexture_dur200ms_lab'],
                    ['20211011_Var6vbsl_set0_im151_camel', '20211011_Var6vbsl_set0_im151_elephant']
                ]
model_names_list = ["rn50_preIN_texture_sizeVar_best", "rn50_preIN_notexture_sizeVar_best", "resnet50-trained-pretrained-vbsl", "rn50_vbsl-dist-ft_epoch15"]
model_names_list = ["SL_resnet50_finetune_vbsl_50k_seed77_model_best", "SL_resnet50_finetune_vbsl_50k_obj_seed777_model_best"]
model_names_list = ["resnet50"]
model_names_list =  ["resnet50", "resnet50_layer1", "resnet50_layer2", "resnet50_layer3", "resnet50_layer4_no_pooling", 'alexnet', 'vggface']

### standard models
# model_name_list = ['resnet50', 'alexnet', 'vggface',  # baseline
#                     'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft', # stylized imagenet
#                     'rn50_preIN_notexture_sizeVar_best', 'rn50_preIN_texture_sizeVar_best', # basel finetuned
#                    'rn50_FromScratch_notexture_sizeVar_best', 'rn50_FromScratch_texture_sizeVar_best', # basel FromScratch
#                     'rn50_preIN_notexture_sizeVar_2losses', # DepthMap prediction
#                    'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'rn50_vbsl-dist-ft_epoch15'] # vbsl
# model_name_list = ['Basel_50k_2id']
# model_name_list = ['vbsl50k_subset_0.5', 'vbsl50k_subset_0.25', 'vbsl50k_subset_0.1', 'vbsl50k_subset_0.05']
### intermediate layers
# model_name_list = ["resnet50_layer1", "resnet50_layer2", "resnet50_layer3", 'resnet50']
# model_name_list = ["resnet50_layer4_no_pooling", "resnet50-trained-pretrained-vbsl_layer1", "resnet50-trained-pretrained-vbsl_layer2", 
#                     "resnet50-trained-pretrained-vbsl_layer3", "resnet50-trained-pretrained-vbsl_layer4_no_pooling"]
# model_name_list = ['Basel_50k_2id']

### 1 epoch ft
# model_name_list = []
# for i in range(41):
#     step = (i+1)*15
#     model_name_list.append(f'vbsl50k_step{step}')
    # model_name_list.append(f'vbsl50kobj_step{step}')

# ## tuning epochs
# model_name_list = ['resnet50']
# for i in range(24):
#     model_name_list.append(f'PennTuning-face-epoch{i}')
#     model_name_list.append(f'PennTuning-obj-epoch{i}')

### intermediate layers
# model_name_list = ["resnet50_layer1", "resnet50_layer2", 
#                     "resnet50_layer3", "resnet50_layer4_no_pooling"]


### pixel
# model_name_list = ['pixel']

# specify which dataset to use by index 
for dataset_index in [0]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):

                
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists

        # fig, ax = plt.subplots(figsize=(9, 8))
        # bars_neptune, bars_elias = [], []

        # Calculate percentages
        # for neptune_count, elias_count in mat_pred_by_bg:
        #     total = neptune_count + elias_count
        #     bars_neptune.append(-(neptune_count / total) * 100)
        #     bars_elias.append((elias_count / total) * 100)

        # ax.barh(unique_bg_name, bars_neptune, color='blue', label='Neptune')
        # ax.barh(unique_bg_name, bars_elias, color='orange', label='Elias')

        # Add text for percentages
        # for i, (neptune_count, elias_count) in enumerate(mat_pred_by_bg):
        #     total = neptune_count + elias_count
        #     neptune_percent, elias_percent = (neptune_count / total) * 100, (elias_count / total) * 100
        #     ax.text(bars_neptune[i], i, f'{neptune_percent:.1f}%', va='center', ha='right', fontsize=15)
        #     ax.text(bars_elias[i], i, f'{elias_percent:.1f}%', va='center', ha='left', fontsize=15)

        # ax.set_xlabel('Percentage', fontsize=17)
        # ax.set_ylabel('Background', fontsize=17)
        # ax.set_title(f'Model Prediction %', fontsize=20)
        # ax.set_xlim(-100, 100)
        # ax.tick_params(axis='y', labelsize=15)
        # ax.legend()
        # plt.show()



In [ ]:
fig, ax = plt.subplots(figsize=(9, 8))
unique_bg_name = ["bg1", "bg2", "bg3", "bg4", "bg5", "bg6", "bg7", "bg8", "bg9", "bg10"]
bars_neptune, bars_elias = [], []

# Calculate percentages
for neptune_count, elias_count in mat_pred_by_bg:
    total = neptune_count + elias_count
    bars_neptune.append(-(neptune_count / total) * 100)
    bars_elias.append((elias_count / total) * 100)

ax.barh(unique_bg_name, bars_neptune, color='blue', label='Neptune')
ax.barh(unique_bg_name, bars_elias, color='orange', label='Elias')

# Add text for percentages
for i, (neptune_count, elias_count) in enumerate(mat_pred_by_bg):
    total = neptune_count + elias_count
    neptune_percent, elias_percent = (neptune_count / total) * 100, (elias_count / total) * 100
    ax.text(bars_neptune[i], i, f'{neptune_percent:.1f}%', va='center', ha='right', fontsize=13)
    ax.text(bars_elias[i], i, f'{elias_percent:.1f}%', va='center', ha='left', fontsize=13)

ax.set_xlabel('Percentage', fontsize=15)
ax.set_ylabel('Background', fontsize=15)
ax.set_title(f'Model Prediction Percentages in {model_name}', fontsize=20)
ax.set_xlim(-100, 100)
ax.tick_params(axis='y', labelsize=15)
ax.legend()
plt.show()

In [ ]:
# Analyze the background bias on vbsl 151

# Background indices 
imageidx_elias = [188,298,1448,16,427,16,427,463,1177,463,188,1177,1406,16,16,298,1406,463,427,463,1448,463,16,188,463,463,1177,463,188,1177,463,1177,1406,298,156,427,1424,1406,1177,1406,427,1424,
            427,1424,16,16,16,1448,16,156,188,1177,463,1424,16,1448,1424,298,463,1424,1448,427,1424,1448,16,16,463,427,1424,16,188, 463,1424,188,298,16,463,188,16,1424,16,463,156, 298,16,1177,
            188,1424,1406,1424,427,1177,427,188,156,188,188,298,1424,156,188,156,156,298,1177,1177,298,1406,188,1424,188,16,1177,156,156,1424,1406,16,427,1406,188,1177,298,16,1424,427,156,427,
            16,298,298,298,1424,463,1448,427,298,1448,156,1448,463,1448,298,298,427,427,427,1448,156,1177, ""]

imageidx_neptune = [1177,1406,188,1177,1406,1406,1424,1406,188,298,298,188,1177,188,427,463,427,188,463,1424,188,298,427,1424,463,16,188,1177,1448,298,1406,427,298,1177,463,427,188,1177,1406,1424,
                    1424,188,1448,16,16,298,188,1406,1424,156,16,1448,156,188,1424,1424,188,463,427,463,298,188,16,188,1424,1177,156,16,1406,156,427,1177,1424,16,298,298,463,16,463,1448,156,1448,16,
                    156,188,1448,298,463,1424,16,1406,1406,1177,1424,1424,156,188,298,298,1448,298,156,1424,1406,16,16,427,1424,1406,463,156,188,1424,463,1424,427,427,16,298,298,427,1424,1177,427,156,
                    427,1424,1424,16,1448,298,1424,1406,463,1406,188,156,1448,427,1406,427,427,1177,463,427,156,156,427,298,1177,""]

assert len(imageidx_elias) == 151 # check the length
assert len(imageidx_neptune) == 151 # check the length

unique_bg = sorted(np.unique(imageidx_elias)[1:].astype(int))

bg_indices_elias = {bg: [] for bg in unique_bg}
bg_indices_neptune = {bg: [] for bg in unique_bg}

# sort backgrounds by indices 
for idx, bg in enumerate(imageidx_elias[:-1]):
    bg_indices_elias[int(bg)].append(idx)

for idx, bg in enumerate(imageidx_neptune[:-1]):
    bg_indices_neptune[int(bg)].append(idx)
    
def extract_pred_by_bg (y_pred, test_indices, unique_bg, imageidx_elias, imageidx_neptune) :
    # Return the proportion of prediction given by the model, by background category
    # y_pred: predictions generated by the model (either 0 or 1) (len = 151)
    # test_indices: indices used for test 
    # bg_indices: dict of indices by backgrounds 

    # extract the bg indices for test indices
    bg_elias, bg_neptune = [imageidx_elias[idx] for idx in test_indices], [imageidx_neptune[idx] for idx in test_indices]
    bg_combined = np.concatenate((bg_elias, bg_neptune))
    # print(bg_combined)
    bg_indices = {bg: [] for bg in unique_bg}
    for idx, bg in enumerate(bg_combined):
        if bg != '' :
            bg_indices[int(bg)].append(idx)
    
    unique_indices = bg_indices.keys()
    model_pred_bg_indices = {bg: [] for bg in unique_indices}

    for idx in unique_indices :
        model_pred = y_pred[bg_indices[idx]]
        unique, counts = np.unique(model_pred, return_counts=True)
        counts_dict = dict(zip(unique, counts))
        # print(counts_dict)
        count_0 = counts_dict.get(0.0, 0)
        count_1 = counts_dict.get(1.0, 0)
        model_pred_bg_indices[idx] = [count_0, count_1]
    return model_pred_bg_indices



In [ ]:
files = sorted(os.listdir("/mnt/smb/locker/issa-locker/users/Seojin/scene_files/zback/"))
selected_files = [files[i] for i in unique_bg if i < len(files)]

# Load and display the images
for i, file in enumerate(selected_files):
    image_path = os.path.join("/mnt/smb/locker/issa-locker/users/Seojin/scene_files/zback/", file)
    fig, ax = plt.subplots(figsize=(2.5, 2.5))
    image = Image.open(image_path)
    ax.imshow(np.array(image), cmap='gray')
    ax.set_title(f"bg{i+1}", fontsize=20)
    ax.axis('off')
    plt.show()


In [ ]:
[imageidx_elias[idx] for idx in test_index]

In [ ]:
# compute i1 and i1_5rep
num_rep = 100
for data_root in useful_stats.keys():
    for model_name in useful_stats[data_root].keys():
        i1_dists = useful_stats[data_root][model_name]['dist']
        avg_i1s = i1_dists.mean(1)
        i1_dists = np.concatenate((i1_dists[0], i1_dists[1]), axis=1)
        model_i1s = []
        for rep_index in range(5):
            np.random.shuffle(i1_dists)
            tmp = np.concatenate([i1_dists[:int(num_rep/2)].mean(0), 
                                  i1_dists[int(num_rep/2):].mean(0)])
            model_i1s.append(np.expand_dims(tmp, axis=0))
        model_i1s = np.concatenate(model_i1s, axis=0)
        
        useful_stats[data_root][model_name]['i1_5rep'] = model_i1s
        useful_stats[data_root][model_name]['i1'] = avg_i1s

        # SL ------------------ Normalize???
        useful_stats[data_root][model_name]['i1_5rep'] = NormalizeData(model_i1s)
        useful_stats[data_root][model_name]['i1'] = NormalizeData(avg_i1s)


In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/']['rn50_preIN_texture_sizeVar_best'].keys()

In [ ]:
import pickle
with open('useful_stats_SL_20240715', 'wb') as f:
    pickle.dump(useful_stats, f)

In [ ]:
import pickle
with open('useful_stats_penn.pkl', 'wb') as f:
    pickle.dump(useful_stats, f)

In [ ]:
# print acc
subset_list = [[0,2], [1,3], [4,5]]
subset_name_list = ['normal_face', 'inverted_face', 'reverse_contrast']
is_print_subset = True

for data_root in useful_stats.keys():
    print(data_root)
    if data_root != '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_101/':

        for model_name in useful_stats[data_root].keys():
            # print acc
            acc = useful_stats[data_root][model_name]['acc']
            print(f"{model_name}: %.4f±%.4f" % (acc[0]*100, acc[1]*100))
            # print subset acc
            tmp = useful_stats[data_root][model_name]['acc_per_img'].mean(0)
            if is_print_subset:
                tmp_group = tmp[:150].reshape(6,25).mean(axis=1)
                for subset_id, subset in enumerate(subset_list):
                    # get acc
                    print("%s acc: %.4f" % (subset_name_list[subset_id], tmp_group[subset].mean()*100))
                    useful_stats[data_root][model_name][f'{subset_name_list[subset_id]}'] = tmp_group[subset].mean()
                print()
        print()

In [ ]:
# # train decoder on vbsl200, and test on vbsli*151
# # vbsli* 151
# data_root = '../../data/face_data/vbsl_151/'
# exp_name_0 = '20211011_Var6vbsl_set0_im151_elias'
# exp_name_1 = '20211011_Var6vbsl_set0_im151_neptune'

# # vbsl 200
# vbsl200_data_root = '../../data/face_data/vbsl_200_new_bg/'
# vbsl200_exp_name_0 = '2022105_Var6vbsl_set0_im200_elias'
# vbsl200_exp_name_1 = '2022105_Var6vbsl_set0_im200_neptune'

# filename_postfix = ''
# num_rep = 100

# model_name_list = ['resnet50', 'alexnet', 'vggface',  # baseline
#                     'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft', # stylized imagenet
#                     'rn50_preIN_notexture_sizeVar_2losses',
#                     'rn50_preIN_notexture_sizeVar_best', # basel - missing 3
#                    'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'rn50_vbsl-dist-ft_epoch15'] # vbsl
# dict_scores = {}
# dict_i1_dists = {}
# dict_conf_mat = {}
# acc_per_img = {}

# for model_name in model_name_list:
#     # load feats for vbsl200
#     _path = os.path.join(vbsl200_data_root, vbsl200_exp_name_0+'_'+model_name+filename_postfix+'.pth')
#     print(f"loading input_1 from {_path}")
#     input_1 = torch.load(_path)
#     _path = os.path.join(vbsl200_data_root, vbsl200_exp_name_1+'_'+model_name+filename_postfix+'.pth')
#     print(f"loading input_2 from {_path}")
#     input_2 = torch.load(_path)
#     input_1 = input_1.reshape(len(input_1), -1)
#     input_2 = input_2.reshape(len(input_2), -1)
#     print(input_1.shape, input_2.shape)
#     output_1 = np.ones(len(input_1))
#     output_2 = np.zeros(len(input_2))
#     vbsl200_inputs = np.concatenate((input_1, input_2), axis=0)
#     vbsl200_outputs = np.concatenate((output_1, output_2), axis=0)
#     # load feats
#     _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
#     print(f"loading input_1 from {_path}")
#     input_1 = torch.load(_path)
#     _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
#     print(f"loading input_2 from {_path}")
#     input_2 = torch.load(_path)
#     input_1 = input_1.reshape(len(input_1), -1)
#     input_2 = input_2.reshape(len(input_2), -1)
#     print(input_1.shape, input_2.shape)
#     output_1 = np.ones(len(input_1))
#     output_2 = np.zeros(len(input_2))
#     inputs = np.concatenate((input_1, input_2), axis=0)
#     outputs = np.concatenate((output_1, output_2), axis=0)

#     half_size = int(vbsl200_inputs.shape[0]/2)
#     X_test, y_test = inputs, outputs
    
#     ## init
#     # [num_class, num_rep, num_img_per_class]
#     # [2, 1000, 101]
#     dists = np.nan * np.ones((2, num_rep, len(output_1))) 
#     all_scores = []
#     acc = np.nan * np.ones((2, num_rep, len(output_1))) 

#     # iter thru num_rep
#     conf_mat = np.zeros((2,2))
#     for rep_index in tqdm(range(num_rep)):
#         # fit model
#         train_idx = np.random.choice(half_size*2, half_size, replace=False)
#         X_train = vbsl200_inputs[train_idx]
#         y_train = vbsl200_outputs[train_idx]
#         clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
#                              tol=1e-4,
#                              fit_intercept=True,
#                              C=1.0,
#                             max_iter = 20000)
#         clf.fit(X_train, y_train)
#         # record score
#         y_predict = clf.predict(X_test)
#         _score = (y_predict == y_test).sum() / len(y_predict)
#         _conf_mat = confusion_matrix(y_test, y_predict)
#         conf_mat += _conf_mat
#         scores.append(_score)
#         _acc_per_img = (y_predict == y_test).astype('float32')
#         acc[0][rep_index] = _acc_per_img[:151]
#         acc[1][rep_index] = _acc_per_img[151:]
#         # record dist
#         _class1_dist = clf.decision_function(input_1)
#         dists[0][rep_index] = _class1_dist
#         _class2_dist = clf.decision_function(input_2)
#         dists[1][rep_index] = _class2_dist * (-1) # negate the dist for second class

#         all_scores.append(np.array(scores).mean())

#     dists = dists/num_rep
#     print(model_name, '%.6f±%.6f'%(np.array(all_scores).mean()*100, np.array(all_scores).std()*100))

#     dict_scores[model_name] = all_scores
#     dict_i1_dists[model_name] = dists
#     conf_mat = conf_mat / num_rep / 2
#     dict_conf_mat[model_name] = conf_mat
#     acc_per_img[model_name] = acc

#     # record useful stats
#     if data_root not in useful_stats:
#         useful_stats[data_root] = {}
#     if model_name not in useful_stats[data_root]:
#         useful_stats[data_root][model_name] = {}

#     useful_stats[data_root][model_name]['dist_vbsl200'] = dists

In [ ]:
model_names_list = ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft'
                    ]

In [ ]:
# (3) exp_names_list_GFR_21

useful_stats = {}
data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151/'
]


exp_name_list = [
                ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_neptune/vbsle151_neptune_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_seojin/vbsle151_seojin_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_sophie/vbsle151_sophie_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_elias/vbsle151_elias_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_seojin/vbsle151_seojin_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_sophie/vbsle151_sophie_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_neptune/vbsle151_neptune_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_sophie/vbsle151_sophie_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_seojin/vbsle151_seojin_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_dan/vbsle151_dan_neutral'],
                 ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_sophie/vbsle151_sophie_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_sreyas/vbsle151_sreyas_neutral'],
                 ['vbsle151_dan/vbsle151_dan_neutral', 'vbsle151_younah/vbsle151_younah_neutral'],
                 ['vbsle151_sreyas/vbsle151_sreyas_neutral', 'vbsle151_younah/vbsle151_younah_neutral']
]
    
# specify which dataset to use by index 
for dataset_index in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18, 19,20]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





In [ ]:
# (5) exp_names_list_GFR_21_texture_colorbg

useful_stats = {}
data_root_list = [
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/',
    '/mnt/smb/locker/issa-locker/users/Seojin/data/face_data/vbsle_151_texture_colorbg/'
]


exp_name_list = [
                ['vbsl151_elias_neutral', 'vbsl151_neptune_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_seojin_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_sophie_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_elias_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_seojin_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_sophie_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_neptune_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_sophie_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_seojin_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_sophie_neutral', 'vbsl151_dan_neutral'],
                 ['vbsl151_sophie_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_sophie_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_dan_neutral', 'vbsl151_sreyas_neutral'],
                 ['vbsl151_dan_neutral', 'vbsl151_younah_neutral'],
                 ['vbsl151_sreyas_neutral', 'vbsl151_younah_neutral']
]

    
# specify which dataset to use by index 
for dataset_index in [0, 1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14,15,16,17, 18, 19,20]:
# for dataset_index in [3,4,5]:
    data_root = data_root_list[dataset_index]
    exp_name_0 = exp_name_list[dataset_index][0]
    exp_name_1 = exp_name_list[dataset_index][1]

    filename_postfix = ''
    num_rep = 100
    
    dict_scores = {}
    dict_i1_dists = {}
    dict_conf_mat = {}
    acc_per_img = {}

    for model_name in model_names_list:
        mat_pred_by_bg = np.zeros((10,2)) # SL 

        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove                                         
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):                                                
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))

        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 151]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                # train_index and test_index does not overlap... and 
                # print(train_index)
                # print(test_index)
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                # print(len(y_test))
                # print(len(test_index))
                # pred_by_bg = extract_pred_by_bg(y_predict, test_index, unique_bg, imageidx_elias, imageidx_neptune)
                # # print(pred_by_bg)
                # mat_pred_by_bg = mat_pred_by_bg + list(pred_by_bg.values())
                _score = (y_predict == y_test).sum() / len(y_predict)
                _conf_mat = confusion_matrix(y_test, y_predict)
                conf_mat += _conf_mat
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist (how far a given sample is from the decision boundary of the classifier)
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class
            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, '%.6f+-%.6f'%(np.array(all_scores).mean(), np.array(all_scores).std()))

        dict_scores[model_name] = all_scores
        dict_i1_dists[model_name] = dists
        conf_mat = conf_mat / num_rep / 2
        dict_conf_mat[model_name] = conf_mat
        acc_per_img[model_name] = acc

        # record useful stats
        if data_root not in useful_stats:
            useful_stats[data_root] = {}
        if model_name not in useful_stats[data_root]:
            useful_stats[data_root][model_name] = {}
        useful_stats[data_root][model_name]['acc'] = [np.array(all_scores).mean(), np.array(all_scores).std()]
        useful_stats[data_root][model_name]['acc_per_img'] = acc.mean(1)
        useful_stats[data_root][model_name]['dist'] = dists





# compare model and bio i1s

In [ ]:
# useful_stats = {}
import pickle
with open('useful_stats_penn.pkl', 'rb') as f:
    useful_stats = pickle.load(f)
    
# hierachy: [task_dir][model_name][acc/acc_per_img/dist/i1]
# acc [list]: acc over all datapoints and trails -> [acc_mean, acc_std]
# acc_per_img [array]: acc per image averaged over all trails -> shape [2, num_img_per_class]
# dist [array]: all svm distance (note that the second class dist is negated) -> shape [2, num_repetition, num_img_per_class]
# i1_5rep [array]: random split-half i1 with 5 repetition -> shape [5, 2*num_img_per_class]
# i1 [array]: model i1 (the same as dist averaging over trails) -> shape [2, num_img_per_class]
# i1_corr [list]: model i1 correlation with bio systems. For vbsl101, the order is [Bourgeois, Sausage, AJ]. 
# For vbsl151, the order is [Human, AJ]. For each corr, I record [i1_corr_mean, i1_corr_std]

In [ ]:
useful_stats.keys()

In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'].keys()

In [ ]:
def subset_upright_5rep_604(array): # (5, 604) to (5, 200)
    return np.concatenate([array[:, 0:25], array[:, 50:75], 
                            array[:, 151:151+25], array[:, 151+50:151+75], 
                            array[:, 302:302+25], array[:, 302+50:302+75], 
                            array[:, 302+151:302+151+25], array[:, 302+151+50:302+151+75], ], axis=1) 

# -------------- SL -----------------
def subset_inverted_5rep_604(array): # (5, 604) to (5, 200)
    return np.concatenate([array[:, 25:50], array[:, 75:100], 
                            array[:, 151+25:151+50], array[:, 151+75:151+100], 
                            array[:, 302+25:302+50], array[:, 302+75:302+100], 
                            array[:, 302+151+25:302+151+50], array[:, 302+151+75:302+151+100], ], axis=1) 

# -------------- SL -----------------
def subset_reverse_contrast_5rep_604(array): # (5, 604) to (5, 200)
    return np.concatenate([array[:, 100:150],  
                            array[:, 151+100:151+150],  
                            array[:, 302+100:302+150], 
                            array[:, 302+151+100:302+151+150], ], axis=1) 

In [ ]:
useful_stats[data_root][model_name]['i1_5rep'].shape

In [ ]:
## compute i1 corr

upright_only = False  # set true to compute for only upright images of vbsli*

bio_i1s_list = [human_i1_5rep, marmoset_i1_5rep]
# bio_i1s_list = [human_i1_5rep]
# bio_i1s_list = [human_i1s_vbsl]
bio_i1s_list = [human_i1s_obj_vbsl151]

data_root = '../../data/face_data/vbsl_151/'

# SL
data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'

data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/'

# model_name_list = ['resnet50_layer1']
# model_name_list = ['resnet50', 'resnet50_layer1'] # SL
# #SL
# model_name_list = ["resnet50", "resnet50_layer1", "resnet50_layer2", 
#                     "resnet50_layer3", "resnet50_layer4_no_pooling"]
# model_name_list = ["rn50_preIN_texture_sizeVar_best", "rn50_preIN_notexture_sizeVar_best", "resnet50-trained-pretrained-vbsl", "rn50_vbsl-dist-ft_epoch15"]

# model_name_list = ['pixel', 'resnet50', 'alexnet', 'vggface',  # baseline
#                     'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft', # stylized imagenet
#                     'rn50_preIN_notexture_sizeVar_best', 'rn50_preIN_texture_sizeVar_best', # basel finetuned
#                    'rn50_FromScratch_notexture_sizeVar_best', 'rn50_FromScratch_texture_sizeVar_best', # basel FromScratch
#                     'rn50_preIN_notexture_sizeVar_2losses', # DepthMap prediction
#                    'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'rn50_vbsl-dist-ft_epoch15'] # vbsl

# model_name_list = ['Basel_50k_2id',]
# model_name_list = []
# for i in range(41):
#     step = (i+1)*15
#     model_name_list.append(f'vbsl50k_step{step}')
#     # model_name_list.append(f'vbsl50kobj_step{step}')

# model_name_list = ['pixel'', resnet50', 'alexnet', 'vggface',  # baseline
#                     'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft', # stylized imagenet
#                     'rn50_preIN_notexture_sizeVar_best', 'rn50_preIN_texture_sizeVar_best', # basel finetuned
#                    'rn50_FromScratch_notexture_sizeVar_best', 'rn50_FromScratch_texture_sizeVar_best', # basel FromScratch
#                     'rn50_preIN_notexture_sizeVar_2losses', # DepthMap prediction
#                    'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'rn50_vbsl-dist-ft_epoch15'] # vbsl
# model_name_list = ['vbsl50k_subset_0.5', 'vbsl50k_subset_0.25', 'vbsl50k_subset_0.1', 'vbsl50k_subset_0.05']
### intermediate layers
# model_name_list = ['resnet50', "resnet50_layer1", "resnet50_layer2", "resnet50_layer3", "resnet50_layer4_no_pooling", 
#                    "resnet50-trained-pretrained-vbsl",
#                        "resnet50-trained-pretrained-vbsl_layer1", "resnet50-trained-pretrained-vbsl_layer2", 
#                     "resnet50-trained-pretrained-vbsl_layer3", "resnet50-trained-pretrained-vbsl_layer4_no_pooling"]
# model_name_list = ['rn50_preIN_texture_sizeVar_best',
#                    'rn50_FromScratch_texture_sizeVar_best', 'rn50_FromScratch_notexture_sizeVar_best']

### tuning epochs
# model_name_list = ['resnet50']
# for i in range(24):
#     model_names_list.append(f'PennTuning-face-epoch{i}')
#     model_name_list.append(f'PennTuning-obj-epoch{i}')

for model_name in model_names_list:
    # get model i1s
    model_i1s = useful_stats[data_root][model_name]['i1_5rep']
    if upright_only:
        model_i1s = subset_upright_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_upright'] = np.zeros((len(bio_i1s_list), 2))
    else:
        useful_stats[data_root][model_name]['i1_corr'] = np.zeros((len(bio_i1s_list), 2))
    for bio_index, bio_i1s in enumerate(bio_i1s_list):
        # get bio i1s
        bio_i1s = bio_i1s.squeeze()
        #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
        bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
        if upright_only:
            bio_i1s = subset_upright_5rep_604(bio_i1s)
        i1_corr = []
        half_length = int(model_i1s.shape[1]/2)
        bio_internal = []
        for rep_index in range(5):
            corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
            corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
            corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
            corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
            print(corr_1, corr_2, corr_3, corr_4, 0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
            i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
            bio_internal.append(corr_4)
        if upright_only:
            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][1] = np.std(np.array(i1_corr))
        else:
            useful_stats[data_root][model_name]['i1_corr'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr'][bio_index][1] = np.std(np.array(i1_corr))

    # ------------------------------- SL -------------------------------
    model_i1s_concat = np.concatenate((useful_stats[data_root][model_name]['i1'][0], useful_stats[data_root][model_name]['i1'][1]))
    useful_stats[data_root][model_name]['i1_corr_diff_per_img'] = [np.zeros(len(model_i1s_concat)), np.zeros(len(model_i1s_concat))]
    for img_index, model_i1_per_img in enumerate(model_i1s_concat) :
            useful_stats[data_root][model_name]['i1_corr_diff_per_img'][0][img_index] = 1 - np.abs(model_i1s_concat[img_index] - human_i1[img_index])
            useful_stats[data_root][model_name]['i1_corr_diff_per_img'][1][img_index] = 1 - np.abs(model_i1s_concat[img_index] - marmoset_i1[img_index])


In [ ]:
compute_upright = True
compute_inverted = True
compute_reverse_contrast = True 

data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'
data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/'
bio_i1s_list = [human_i1_5rep, marmoset_i1_5rep]
# bio_i1s_list = [human_i1_5rep]
# bio_i1s_list = [human_i1s_vbsl]
bio_i1s_list = [human_i1s_obj_vbsl151]

# i1 corr - upright only 
for model_name in model_names_list:
    # get model i1s
    if compute_upright :
        model_i1s = useful_stats[data_root][model_name]['i1_5rep'] 
        model_i1s = subset_upright_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_upright'] = np.zeros((len(bio_i1s_list), 2))
        for bio_index, bio_i1s in enumerate(bio_i1s_list):
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
            bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
            bio_i1s = subset_upright_5rep_604(bio_i1s)
            print(model_name, bio_index)
            i1_corr = []
            half_length = int(model_i1s.shape[1]/2)
            bio_internal = []
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                print(corr_1, corr_2, corr_3, corr_4, 0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                bio_internal.append(corr_4)

            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_upright'][bio_index][1] = np.std(np.array(i1_corr))

    if compute_inverted : 
        model_i1s = useful_stats[data_root][model_name]['i1_5rep']
        model_i1s = subset_inverted_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_inverted'] = np.zeros((len(bio_i1s_list), 2))

        for bio_index, bio_i1s in enumerate(bio_i1s_list):
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
            bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
            bio_i1s = subset_inverted_5rep_604(bio_i1s)
            print(model_name, bio_index)
            i1_corr = []
            half_length = int(model_i1s.shape[1]/2)
            bio_internal = []
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                print(corr_1, corr_2, corr_3, corr_4)
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                bio_internal.append(corr_4)


            # print(np.mean(np.array(bio_internal)), np.std(np.array(bio_internal)))
    
            useful_stats[data_root][model_name]['i1_corr_inverted'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_inverted'][bio_index][1] = np.std(np.array(i1_corr))

    if compute_reverse_contrast : 
        model_i1s = useful_stats[data_root][model_name]['i1_5rep']
        model_i1s = subset_reverse_contrast_5rep_604(model_i1s)
        useful_stats[data_root][model_name]['i1_corr_reverse_contrast'] = np.zeros((len(bio_i1s_list), 2))
        for bio_index, bio_i1s in enumerate(bio_i1s_list):
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            #bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only # SL
            bio_i1s = np.nan_to_num(bio_i1s) # for monkey_i1s_3 only
            bio_i1s = subset_reverse_contrast_5rep_604(bio_i1s)
            print(model_name, bio_index)
            i1_corr = []
            half_length = int(model_i1s.shape[1]/2)
            bio_internal = []
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                print(corr_1, corr_2, corr_3, corr_4, 0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                bio_internal.append(corr_4)

            # print(np.mean(np.array(bio_internal)), np.std(np.array(bio_internal)))
    
            useful_stats[data_root][model_name]['i1_corr_reverse_contrast'][bio_index][0] = np.mean(np.array(i1_corr))
            useful_stats[data_root][model_name]['i1_corr_reverse_contrast'][bio_index][1] = np.std(np.array(i1_corr))


In [ ]:
# save
import pickle
with open('useful_stats_SL_20240723.pkl', 'wb') as f:
    pickle.dump(useful_stats, f)

In [ ]:
# save
import pickle
with open('useful_stats_penn.pkl', 'wb') as f:
    pickle.dump(useful_stats, f)

In [ ]:
i1_corr

In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/']['SL_resnet50_finetune_vbsl_50k_seed77_model_best']['i1_corr_reverse_contrast']

In [ ]:
# print acc and i1s
# data_root = '../../data/face_data/vbsl_151/'
data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'
data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/control_exp/objs/'

# model_name_list = ['resnet50', 'alexnet', 'vggface',  # baseline
#                     'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft', # stylized imagenet
#                     'rn50_preIN_notexture_sizeVar_best', 'rn50_preIN_texture_sizeVar_best', # basel finetuned
#                    'rn50_FromScratch_notexture_sizeVar_best', 'rn50_FromScratch_texture_sizeVar_best', # basel FromScratch
#                     'rn50_preIN_notexture_sizeVar_2losses', # DepthMap prediction
#                    'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'rn50_vbsl-dist-ft_epoch15'] # vbsl
# model_name_list = ['vbsl50k_subset_0.5', 'vbsl50k_subset_0.25', 'vbsl50k_subset_0.1', 'vbsl50k_subset_0.05']
# model_name_list = ['Basel_50k_2id',]
# model_name_list = []
# for i in range(41):
#     step = (i+1)*15
#     model_name_list.append(f'vbsl50k_step{step}')
#     # model_name_list.append(f'vbsl50kobj_step{step}')
# model_name_list = ['pixel']
# model_name_list = ['resnet50', 'resnet50_layer1'] # SL
# model_name_list = ["resnet50", "resnet50_layer1", "resnet50_layer2", 
#                     "resnet50_layer3", "resnet50_layer4_no_pooling"]
# model_name_list = ["rn50_preIN_texture_sizeVar_best", "rn50_preIN_notexture_sizeVar_best", "resnet50-trained-pretrained-vbsl", "rn50_vbsl-dist-ft_epoch15"]

acc_list = []
acc_std_list = []


i1_corr_human_list = []
i1_corr_human_list_upright, i1_corr_human_list_inverted, i1_corr_human_list_reverse_contrast = [], [], []

i1_corr_std_human_list = []
i1_corr_std_human_list_upright, i1_corr_std_human_list_inverted, i1_corr_std_human_list_reverse_contrast = [], [], []

i1_corr_AJ_list = []
i1_corr_AJ_list_upright, i1_corr_AJ_list_inverted, i1_corr_AJ_list_reverse_contrast = [], [], []

i1_corr_std_AJ_list = []
i1_corr_std_AJ_list_upright, i1_corr_std_AJ_list_inverted, i1_corr_std_AJ_list_reverse_contrast = [], [], []

for model_name in model_names_list:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_corr_upright = useful_stats[data_root][model_name]['i1_corr_upright']
    i1_corr_inverted = useful_stats[data_root][model_name]['i1_corr_inverted']
    i1_corr_reverse_contrast = useful_stats[data_root][model_name]['i1_corr_reverse_contrast']
    print(i1_corr_reverse_contrast)
    acc = useful_stats[data_root][model_name]['acc']
    acc_list.append(acc[0])
    acc_std_list.append(acc[1])
    # print stats
    print(f"{model_name}")
    print("acc: %.6f±%.6f" % (acc[0],acc[1]))
    for i1_index, cur_i1 in enumerate(i1_corr):
        print(f"i1 #{i1_index}: %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list.append(cur_i1[0])
            i1_corr_std_human_list.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list.append(cur_i1[0])
            i1_corr_std_AJ_list.append(cur_i1[1])
    # ------------------ SL --------------------------
    for i1_index, cur_i1 in enumerate(i1_corr_upright):
        print(f"i1 #{i1_index} (upright): %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list_upright.append(cur_i1[0])
            i1_corr_std_human_list_upright.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list_upright.append(cur_i1[0])
            i1_corr_std_AJ_list_upright.append(cur_i1[1])
    for i1_index, cur_i1 in enumerate(i1_corr_inverted):
        print(f"i1 #{i1_index} (inverted): %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list_inverted.append(cur_i1[0])
            i1_corr_std_human_list_inverted.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list_inverted.append(cur_i1[0])
            i1_corr_std_AJ_list_inverted.append(cur_i1[1])
    for i1_index, cur_i1 in enumerate(i1_corr_reverse_contrast):
        print(f"i1 #{i1_index} (reverse contrast): %.6f±%.6f" % (cur_i1[0],cur_i1[1]))
        if i1_index == 0:
            i1_corr_human_list_reverse_contrast.append(cur_i1[0])
            i1_corr_std_human_list_reverse_contrast.append(cur_i1[1])
        elif i1_index == 1:
            i1_corr_AJ_list_reverse_contrast.append(cur_i1[0])
            i1_corr_std_AJ_list_reverse_contrast.append(cur_i1[1])
    # ------------------------------------------------
       
    # print subset acc
    tmp = useful_stats[data_root][model_name]['acc_per_img'].mean(0)
    if is_print_subset:
        tmp_group = tmp[:150].reshape(6,25).mean(axis=1)
        for subset_id, subset in enumerate(subset_list):
            # get acc
            print("%s acc: %.4f" % (subset_name_list[subset_id], tmp_group[subset].mean()*100))
    print()
    # print(f"{model_name} \n%.2f/%.2f/%.2f" % (i1_corr[0][0], i1_corr[1][0], i1_corr[2][0]))

In [ ]:
# plot for IDs ablation
results_acc = np.array([[66.8236, 62.4998, 62.3229],
                       [65.0423, 63.9602, 63.9973],
                       [63.2063, 64.2012, 61.3207],
                       [64.2957, 66.7939, 62.9402]])
results_i1s = np.array([[0.000691, 0.021171, 0.023820],
                       [0.062417, 0.070766, 0.108321],
                       [0.105385, 0.058539, 0.005559],
                       [0.136031, 0.195483, 0.081920]])
xx = ["2","4","6","8"]
plt.errorbar(xx, results_acc.mean(1), yerr=results_acc.std(1))
plt.title("Num IDs Ablation")
plt.xlabel("Num of IDs")
plt.ylabel("acc")
plt.show()

xx = ["2","4","6","8"]
plt.errorbar(xx, results_i1s.mean(1), yerr=results_i1s.std(1))
plt.title("Num IDs Ablation")
plt.xlabel("Num of IDs")
plt.ylabel("i1s")
plt.show()

In [ ]:
acc_list

In [ ]:
model_names_list

In [ ]:
# SL
models = ["resnet50", "resnet50-layer1", "resnet50-layer2", "resnet50-layer3", "resnet50-layer4-no-pooling"]
models = ["finetuned-Basel", "finetuned-Basel-NoTexture", "finetuned-AppleFaces", "finetuned-AppleFaces-dist"]
models = ["finetuned-AppleFaces50k", "finetuned-Object50k", "vanilla resnet"]
# models = []

# for model_name in model_names_list :
#     models.append(model_name.split("-")[-1])

plt.figure(figsize=(8, 8))  
acc_list = [69.7722/100, 85.6704/100,72.5794/100]
acc_std_list = [2.2411/100, 1.4935/100, 2.2267/100]
bars = plt.bar(models, acc_list, yerr=acc_std_list, capsize=5)

# Add text annotations for each bar
for bar, acc in zip(bars, acc_list):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)

plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("accuracy", fontsize=20)

# Set the range of y-axis from 0 to 1
plt.ylim(0.0, 1.05)

plt.legend()
plt.title("ResNet50 from scratch, accuracy", fontsize=20)
plt.title("ResNet50, tested on GOR vbsli*151", fontsize=20)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
models = ["finetuned-AppleFaces50k", "finetuned-Object50k", "vanilla resnet50"]
# Dummy data for demonstration purposes
# Define the position of the bars
x = np.arange(len(models))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(11, 9))
acc_list_upright = [val / 100 for val in [71.23,97.57,75.241]]
                    # [71.23,97.57,74.38]]
acc_list_inverted = [val / 100 for val in [70.83,83.6,71.79]]
    # [70.83,83.6,71.2]]
acc_list_reverse_contrast = [val / 100 for val in [67.55,75.61,70.56]]
# [67.55,75.61,70.22]]

# Plotting each set of bars
bars0 = ax.bar(x - width * 1.5, acc_list, width,label='all', color='blue', alpha=1.0, capsize=5)
bars1 = ax.bar(x - width * 0.5 , acc_list_upright, width,label='upright', color='blue', alpha=0.65, capsize=5)
bars2 = ax.bar(x +  width * 0.5, acc_list_inverted, width,  label='inverted', color='blue', alpha=0.3, capsize=5)
bars3 = ax.bar(x + width * 1.5, acc_list_reverse_contrast, width, label='reverse contrast', color='blue', alpha=0.1, capsize=5)

# Add text annotations for each bar
for bars in [bars0, bars1, bars2, bars3]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=15)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Models', fontsize=18)
ax.set_ylabel('Accuracy', fontsize=18)
ax.set_title('Fine-tuned and vanilla ResNet50, tested on GOR vbsli*151', fontsize=22)
ax.set_xticks(x)
ax.set_xticklabels([label.replace('-', '-\n') for label in models], fontsize=18)
ax.yaxis.set_tick_params(labelsize=15)

ax.legend(fontsize=15, bbox_to_anchor=(1.05, 1), loc='upper left')


# Display the plot
plt.ylim(0.0, 1.05)
plt.show()


In [ ]:
# SL

acc_list_upright = [val/100 for val in [98.65,65.91]]
acc_std_list_upright = []
acc_list_inverted = [val/100 for val in [76.39,61.16]]
acc_std_list_inverted = []
acc_list_reverse_contrast = [val/100 for val in [82.37,52.9]]
acc_std_list_reverse_contrast = []


plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, acc_list_upright, capsize=5)
# Add text annotations for each bar
for bar, acc in zip(bars, acc_list_upright):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("acc", fontsize=20)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50 (upright only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, acc_list_inverted, capsize=5)
# Add text annotations for each bar
for bar, acc in zip(bars, acc_list_inverted):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

# plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("acc", fontsize=20)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50 (inverted only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, acc_list_reverse_contrast, capsize=5)
# Add text annotations for each bar
for bar, acc in zip(bars, acc_list_reverse_contrast):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=20)
# plt.xlabel("steps")
plt.ylabel("acc", fontsize=20)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, (reverse contrast only)", fontsize=20)
plt.show()

In [ ]:
models

In [ ]:
# SL
plt.figure(figsize=(7, 8)) 
i1_corr_human_list = [0.356536,0.398955,0.418159]
i1_corr_std_human_list = [0.023619, 0.020221, 0.021829]
# [0.023619, 0.020221, 0.023948]
bars = plt.bar(models, i1_corr_human_list, yerr=i1_corr_std_human_list, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

#plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=15)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(-0.15, 1.05)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GOR vbsli* 151", fontsize=20)
plt.show()

# ---------------------------- SL -------------------------------------
plt.figure(figsize=(7, 8))  

bars = plt.bar(models, i1_corr_AJ_list, yerr=i1_corr_std_AJ_list, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_AJ_list):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=20)

#plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=18)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)

plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, marmoset i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GOR vbsli* 151", fontsize=20)
plt.show()

In [ ]:
# SL
i1_corr_human_list_upright = [0.42481,0.512017,0.349363]
# [0.424810,0.512017,0.347575]
i1_corr_std_human_list_upright = [0.472004,0.392035,0.493194]
# [0.035555, 0.030237, 0.031485]
i1_corr_human_list_inverted = [0.472004,0.392035,0.493194]
# [0.035555, 0.030237, 0.031485]
i1_corr_std_human_list_inverted = [0.015472,0.037951,0.019945]
# [0.034771,0.033141, 0.029989]
i1_corr_human_list_reverse_contrast = [0.306668,0.319522,0.416907]
i1_corr_std_human_list_reverse_contrast = [0.007847, 0.017272, 0.011585]
# [0.010393,0.007654,0.014628]

plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, i1_corr_human_list_upright, yerr=i1_corr_std_human_list_upright, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list_upright):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=15)

#plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right', fontsize=15)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)

plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GOR\nvbsli*151, (upright only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, i1_corr_human_list_inverted, yerr=i1_corr_std_human_list_inverted, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list_inverted):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=15)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GFR\nvbsli*151, (inverted only)", fontsize=20)
plt.show()

# -------------------------------------------------------------------
plt.figure(figsize=(6, 8)) 

bars = plt.bar(models, i1_corr_human_list_reverse_contrast, yerr=i1_corr_std_human_list_reverse_contrast, capsize=5, color='green')
# Add text annotations for each bar
for bar, acc in zip(bars, i1_corr_human_list_reverse_contrast):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() * 0.6 , yval, f'{acc:.2f}', ha='left', va='bottom', fontsize=15)

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], fontsize=18)
plt.yticks(fontsize=15)
plt.xlabel("model", fontsize=18)
# plt.xlabel("steps")
plt.ylabel("i1", fontsize=18)
plt.ylim(0.0, 1)
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr", fontsize=20)
plt.title("Finetuned ResNet50, tested on GFR\nvbsli*151, (reverse contrast only)", fontsize=20)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
models = ["finetuned-AppleFaces50k", "finetuned-Object50k", "vanilla resnet"]
i1_corr_human_list = [0.356536,0.398955,0.418159]
# Define the position of the bars
x = np.arange(len(models))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(10, 9))

# Plotting each set of bars
bars0 = ax.bar(x - width * 1.5, i1_corr_human_list, width,label='all', color='green', alpha=1.0, capsize=5)
bars1 = ax.bar(x - width * 0.5 , i1_corr_human_list_upright, width,label='upright', color='green', alpha=0.65, capsize=5)
bars2 = ax.bar(x +  width * 0.5, i1_corr_human_list_inverted, width,  label='inverted', color='green', alpha=0.3, capsize=5)
bars3 = ax.bar(x + width * 1.5, i1_corr_human_list_reverse_contrast, width, label='reverse contrast', color='green', alpha=0.1, capsize=5)

# Add text annotations for each bar
for bars in [bars0, bars1, bars2, bars3]:
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2f}', ha='center', va='bottom', fontsize=18)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Models', fontsize=18)
ax.set_ylabel('i1', fontsize=18)
ax.set_title('Finetuned ResNet50, tested on GOR vbsli*151', fontsize=22)
ax.set_xticks(x)

ax.set_xticklabels([label.replace('-', '-\n') for label in models], fontsize=18)
ax.yaxis.set_tick_params(labelsize=15)

ax.legend(fontsize=15, bbox_to_anchor=(1.05, 1), loc='upper left')


# Display the plot
plt.ylim(-0.15, 1.05)
plt.show()


In [ ]:
# SL


plt.errorbar(models, i1_corr_human_list, yerr=i1_corr_std_human_list)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right')

plt.xlabel("model")
plt.xlabel("steps")
plt.ylabel("i1")
plt.legend()
plt.title("ResNet50 from scratch, human i1 corr")
plt.title("AppleFaces (no texture), human i1 corr")

plt.show()

# SL

plt.errorbar(models, i1_corr_AJ_list, yerr=i1_corr_std_AJ_list)
plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right')

plt.xlabel("model")
plt.xlabel("steps")
plt.ylabel("i1")
plt.legend()
plt.title("ResNet50 from scratch, marmoset i1 corr")
plt.title("AppleFaces (no texture), marmoset i1 corr")

plt.show()

In [ ]:

plt.errorbar(models, i1_corr_human_list, yerr=i1_corr_std_human_list, label='human_i1', color='blue')
plt.errorbar(models, i1_corr_AJ_list, yerr=i1_corr_std_AJ_list, label='marmoset_i1', color='orange')

plt.xticks(ticks=np.arange(len(models)), labels=[label.replace('-', '-\n') for label in models], rotation=45, ha='right')

plt.xlabel("model")
plt.xlabel("steps")
plt.ylabel("i1")
plt.legend()
plt.title("AppleFaces (no texture), human and marmoset i1 corr")

plt.show()

### SL - Analysis

In [ ]:
elias_acc = useful_stats[data_root][model_name]['acc_per_img'][0]
neptune_acc = useful_stats[data_root][model_name]['acc_per_img'][1]
combined_acc = np.concatenate((elias_acc, neptune_acc))

acc_top_idx = np.argsort(combined_acc)[-20:][::-1]
acc_top_val = combined_acc[acc_top_idx]
acc_top = list(zip(acc_top_idx, acc_top_val))
acc_top

In [ ]:
useful_stats_top_bottom

In [ ]:
np.min(human_i1)

In [ ]:
## vbsl151 -- Highest and lowest accuracy per model 
useful_stats_top_bottom = {}
data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'
human_i1_top_idx, human_i1_bottom_idx = np.argsort(human_i1)[-15:][::-1], np.argsort(human_i1)[:15]
human_i1_top_val, human_i1_bottom_val  = human_i1[human_i1_top_idx], human_i1[human_i1_bottom_idx]
human_i1_top, human_i1_bottom = list(zip(human_i1_top_idx, human_i1_top_val)), list(zip(human_i1_bottom_idx, human_i1_bottom_val))

for model_name in ["human"] :
    useful_stats_top_bottom[model_name] = {
        'top' : {
            'acc' : [[],[]],
            'acc_u' : [[],[]],
            'acc_i' : [[],[]],
            'acc_rc' : [[],[]],
            'human_i1' : [[],[]],
            'marmoset_i1' : [[],[]],
        },
        'bottom'  : {
            'acc' : [[],[]],
            'acc_u' : [[],[]],
            'acc_i' : [[],[]],
            'acc_rc' : [[],[]],
            'human_i1' : [[],[]],
            'marmoset_i1' : [[],[]],
        }
    }


    combined_idx = np.arange(0, 302)
    combined_idx_u = np.concatenate([np.arange(0, 25), np.arange(50,75), np.arange(151,151+25), np.arange(151+50, 151+75)])
    combined_idx_i = np.concatenate([np.arange(25, 50), np.arange(75,100), np.arange(151+25,151+50), np.arange(151+75, 151+100)])
    combined_idx_rc = np.concatenate([np.arange(100, 150), np.arange(151+100,151+150)])

    human_i1_u = human_i1[combined_idx_u]
    human_i1_i = human_i1[combined_idx_i]
    human_i1_rc = human_i1[combined_idx_rc]

    random.Random(4).shuffle(combined_idx)
    random.Random(4).shuffle(combined_acc)
    

    acc_top_idx, acc_bottom_idx = combined_idx[np.argsort(human_i1)[-15:][::-1]], combined_idx[np.argsort(human_i1)[:15]]
    acc_top_idx_u, acc_bottom_idx_u = combined_idx_u[np.argsort(human_i1_u)[-5:][::-1]], combined_idx_u[np.argsort(human_i1_u)[:5]]
    acc_top_idx_i, acc_bottom_idx_i = combined_idx_i[np.argsort(human_i1_i)[-5:][::-1]], combined_idx_i[np.argsort(human_i1_i)[:5]]
    acc_top_idx_rc, acc_bottom_idx_rc = combined_idx_rc[np.argsort(human_i1_rc)[-5:][::-1]], combined_idx_rc[np.argsort(human_i1_rc)[:5]]

    acc_top_val, acc_bottom_val = human_i1[acc_top_idx], human_i1[acc_bottom_idx]
    acc_top_val_u, acc_bottom_val_u = human_i1[acc_top_idx_u], human_i1[acc_bottom_idx_u]
    acc_top_val_i, acc_bottom_val_i = human_i1[acc_top_idx_i], human_i1[acc_bottom_idx_i]
    acc_top_val_rc, acc_bottom_val_rc = human_i1[acc_top_idx_rc], human_i1[acc_bottom_idx_rc]

    acc_top, acc_bottom = list(zip(acc_top_idx, acc_top_val)), list(zip(acc_bottom_idx, acc_bottom_val))
    acc_top_u, acc_bottom_u = list(zip(acc_top_idx_u, acc_top_val_u)), list(zip(acc_bottom_idx_u, acc_bottom_val_u))
    acc_top_i, acc_bottom_i = list(zip(acc_top_idx_i, acc_top_val_i)), list(zip(acc_bottom_idx_i, acc_bottom_val_i))
    acc_top_rc, acc_bottom_rc = list(zip(acc_top_idx_rc, acc_top_val_rc)), list(zip(acc_bottom_idx_rc, acc_bottom_val_rc))

    useful_stats_top_bottom[model_name]['top']['acc'] = acc_top
    useful_stats_top_bottom[model_name]['top']['acc_u'] = acc_top_u
    useful_stats_top_bottom[model_name]['top']['acc_i'] = acc_top_i
    useful_stats_top_bottom[model_name]['top']['acc_rc'] = acc_top_rc
    useful_stats_top_bottom[model_name]['bottom']['acc'] = acc_bottom
    useful_stats_top_bottom[model_name]['bottom']['acc_u'] = acc_bottom_u
    useful_stats_top_bottom[model_name]['bottom']['acc_i'] = acc_bottom_i
    useful_stats_top_bottom[model_name]['bottom']['acc_rc'] = acc_bottom_rc

from PIL import Image as PILImage


# Function to get the image paths based on the index
def get_image_path(idx):
    if idx < 151:
        return f"/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_elias/trialnum{idx}.png"
    else:
        return f"/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_neptune/trialnum{idx-151}.png"

# Displaying the images
for model_name in ["human"]:
    # Get the top 10 and bottom 10 image paths
    
    top_image_paths = [get_image_path(idx) for idx, _ in human_i1_top]
    bottom_image_paths = [get_image_path(idx) for idx, _ in human_i1_bottom]

    # Concatenate the top 10 images
    top_concatenated_image = concatenate_images(top_image_paths)
    plt.figure(figsize=(25, 8))
    plt.imshow(top_concatenated_image)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths)} images for human", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths, human_i1_top)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, top_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # Concatenate the bottom 10 images
    bottom_concatenated_image = concatenate_images(bottom_image_paths)
    plt.figure(figsize=(25, 8))
    plt.imshow(bottom_concatenated_image)
    plt.axis('off')
    plt.title(f"Bottom {len(top_image_paths)} images for human", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths, human_i1_bottom)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, bottom_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    print(model_name)
    acc_top_u, acc_top_i, acc_top_rc = useful_stats_top_bottom[model_name]['top']['acc_u'], useful_stats_top_bottom[model_name]['top']['acc_i'], useful_stats_top_bottom[model_name]['top']['acc_rc']
    acc_bottom_u, acc_bottom_i, acc_bottom_rc = useful_stats_top_bottom[model_name]['bottom']['acc_u'], useful_stats_top_bottom[model_name]['bottom']['acc_i'], useful_stats_top_bottom[model_name]['bottom']['acc_rc']

    top_image_paths_u, top_image_paths_i, top_image_paths_rc  = [get_image_path(idx) for idx, _ in acc_top_u], [get_image_path(idx) for idx, _ in acc_top_i], [get_image_path(idx) for idx, _ in acc_top_rc]
    bottom_image_paths_u, bottom_image_paths_i, bottom_image_paths_rc  = [get_image_path(idx) for idx, _ in acc_bottom_u], [get_image_path(idx) for idx, _ in acc_bottom_i], [get_image_path(idx) for idx, _ in acc_bottom_rc]

    acc_top_u_concat, acc_top_i_concat, acc_top_rc_concat = concatenate_images(top_image_paths_u), concatenate_images(top_image_paths_i), concatenate_images(top_image_paths_rc)
    acc_bottom_u_concat, acc_bottom_i_concat, acc_bottom_rc_concat = concatenate_images(bottom_image_paths_u), concatenate_images(bottom_image_paths_i), concatenate_images(bottom_image_paths_rc)
    # upright
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_top_u_concat)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths_u)} upright for {model_name}", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths_u, acc_top_u)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_top_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # inverted
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_top_i_concat)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths_i)} inverted for {model_name}", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths_i, acc_top_i)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_top_i_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # reverse contrast
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_top_rc_concat)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths_rc)} reverse contrast for {model_name}", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths_rc, acc_top_rc)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_top_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    #-------------- BOTTOM -------------------
    # upright
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_bottom_u_concat)
    plt.axis('off')
    plt.title(f"Bottom {len(bottom_image_paths_u)} upright for {model_name}", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths_u, acc_bottom_u)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_bottom_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # inverted
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_bottom_i_concat)
    plt.axis('off')
    plt.title(f"Bottom {len(bottom_image_paths_i)} inverted for {model_name}", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths_i, acc_bottom_i)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_bottom_i_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # reverse contrast
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_bottom_rc_concat)
    plt.axis('off')
    plt.title(f"Bottom {len(bottom_image_paths_rc)} reverse contrast for {model_name}", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths_rc, acc_bottom_rc)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_bottom_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()




In [ ]:
useful_stats_top_bottom['rn50_preIN_texture_sizeVar_best']['top']['acc_u'] 

In [ ]:
## vbsl151 -- Highest and lowest accuracy per model 
import random

useful_stats_top_bottom = {}
model_names_list = ["rn50_preIN_texture_sizeVar_best", "rn50_preIN_notexture_sizeVar_best", "resnet50-trained-pretrained-vbsl", "rn50_vbsl-dist-ft_epoch15"]

data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'
for model_name in model_names_list :
    useful_stats_top_bottom[model_name] = {
        'top' : {
            'acc' : [[],[]],
            'acc_u' : [[],[]],
            'acc_i' : [[],[]],
            'acc_rc' : [[],[]],
            'human_i1' : [[],[]],
            'marmoset_i1' : [[],[]],
        },
        'bottom'  : {
            'acc' : [[],[]],
            'acc_u' : [[],[]],
            'acc_i' : [[],[]],
            'acc_rc' : [[],[]],
            'human_i1' : [[],[]],
            'marmoset_i1' : [[],[]],
        }
    }

    elias_acc = useful_stats[data_root][model_name]['acc_per_img'][0]
    neptune_acc = useful_stats[data_root][model_name]['acc_per_img'][1]
    combined_idx = np.arange(0, 302)
    combined_idx_u = np.concatenate([np.arange(0, 25), np.arange(50,75), np.arange(151,151+25), np.arange(151+50, 151+75)])
    combined_idx_i = np.concatenate([np.arange(25, 50), np.arange(75,100), np.arange(151+25,151+50), np.arange(151+75, 151+100)])
    combined_idx_rc = np.concatenate([np.arange(100, 150), np.arange(151+100,151+150)])

    combined_acc = np.concatenate((elias_acc, neptune_acc))
    combined_acc_u = combined_acc[combined_idx_u]
    combined_acc_i = combined_acc[combined_idx_i]
    combined_acc_rc = combined_acc[combined_idx_rc]

    random.Random(4).shuffle(combined_idx)
    random.Random(4).shuffle(combined_acc)
    

    acc_top_idx, acc_bottom_idx = combined_idx[np.argsort(combined_acc)[-15:][::-1]], combined_idx[np.argsort(combined_acc)[:15]]
    acc_top_idx_u, acc_bottom_idx_u = combined_idx_u[np.argsort(combined_acc_u)[-5:][::-1]], combined_idx_u[np.argsort(combined_acc_u)[:5]]
    acc_top_idx_i, acc_bottom_idx_i = combined_idx_i[np.argsort(combined_acc_i)[-5:][::-1]], combined_idx_i[np.argsort(combined_acc_i)[:5]]
    acc_top_idx_rc, acc_bottom_idx_rc = combined_idx_rc[np.argsort(combined_acc_rc)[-5:][::-1]], combined_idx_rc[np.argsort(combined_acc_rc)[:5]]

    acc_top_val, acc_bottom_val = np.concatenate((elias_acc, neptune_acc))[acc_top_idx], np.concatenate((elias_acc, neptune_acc))[acc_bottom_idx]
    acc_top_val_u, acc_bottom_val_u = np.concatenate((elias_acc, neptune_acc))[acc_top_idx_u], np.concatenate((elias_acc, neptune_acc))[acc_bottom_idx_u]
    acc_top_val_i, acc_bottom_val_i = np.concatenate((elias_acc, neptune_acc))[acc_top_idx_i], np.concatenate((elias_acc, neptune_acc))[acc_bottom_idx_i]
    acc_top_val_rc, acc_bottom_val_rc = np.concatenate((elias_acc, neptune_acc))[acc_top_idx_rc], np.concatenate((elias_acc, neptune_acc))[acc_bottom_idx_rc]

    acc_top, acc_bottom = list(zip(acc_top_idx, acc_top_val)), list(zip(acc_bottom_idx, acc_bottom_val))
    acc_top_u, acc_bottom_u = list(zip(acc_top_idx_u, acc_top_val_u)), list(zip(acc_bottom_idx_u, acc_bottom_val_u))
    acc_top_i, acc_bottom_i = list(zip(acc_top_idx_i, acc_top_val_i)), list(zip(acc_bottom_idx_i, acc_bottom_val_i))
    acc_top_rc, acc_bottom_rc = list(zip(acc_top_idx_rc, acc_top_val_rc)), list(zip(acc_bottom_idx_rc, acc_bottom_val_rc))

    useful_stats_top_bottom[model_name]['top']['acc'] = acc_top
    useful_stats_top_bottom[model_name]['top']['acc_u'] = acc_top_u
    useful_stats_top_bottom[model_name]['top']['acc_i'] = acc_top_i
    useful_stats_top_bottom[model_name]['top']['acc_rc'] = acc_top_rc
    useful_stats_top_bottom[model_name]['bottom']['acc'] = acc_bottom
    useful_stats_top_bottom[model_name]['bottom']['acc_u'] = acc_bottom_u
    useful_stats_top_bottom[model_name]['bottom']['acc_i'] = acc_bottom_i
    useful_stats_top_bottom[model_name]['bottom']['acc_rc'] = acc_bottom_rc


from PIL import Image as PILImage

def concatenate_images(image_paths):
    images = [PILImage.open(img_path) for img_path in image_paths]
    widths, heights = zip(*(img.size for img in images))

    total_width = sum(widths)
    max_height = max(heights)

    concatenated_image = PILImage.new('RGB', (total_width, max_height))

    x_offset = 0
    for img in images:
        concatenated_image.paste(img, (x_offset, 0))
        x_offset += img.width

    return concatenated_image

# Function to get the image paths based on the index
def get_image_path(idx):
    if idx < 151:
        return f"/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_elias/trialnum{idx}.png"
    else:
        return f"/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_neptune/trialnum{idx-151}.png"

# Displaying the images
for model_name in model_names_list:
    # # Get the top 10 and bottom 10 image paths
    # acc_top = useful_stats_top_bottom[model_name]['top']['acc']
    # acc_bottom = useful_stats_top_bottom[model_name]['bottom']['acc']

    # top_image_paths = [get_image_path(idx) for idx, _ in acc_top]
    # bottom_image_paths = [get_image_path(idx) for idx, _ in acc_bottom]

    # # Concatenate the top 10 images
    # top_concatenated_image = concatenate_images(top_image_paths)
    # plt.figure(figsize=(25, 8))
    # plt.imshow(top_concatenated_image)
    # plt.axis('off')
    # plt.title(f"Top {len(top_image_paths)} images for {model_name}")

    # # Add accuracy values below each image for top images
    # x_offset = 0
    # for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths, acc_top)):
    #     img_width = PILImage.open(image_path).width
    #     plt.text(x_offset + img_width / 2, top_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
    #     x_offset += img_width

    # plt.show()

    # # Concatenate the bottom 10 images
    # bottom_concatenated_image = concatenate_images(bottom_image_paths)
    # plt.figure(figsize=(25, 8))
    # plt.imshow(bottom_concatenated_image)
    # plt.axis('off')
    # plt.title(f"Bottom {len(top_image_paths)} images for {model_name}")

    # # Add accuracy values below each image for bottom images
    # x_offset = 0
    # for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths, acc_bottom)):
    #     img_width = PILImage.open(image_path).width
    #     plt.text(x_offset + img_width / 2, bottom_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
    #     x_offset += img_width

    # plt.show()

    # Concatnete the top 5 images per category (upright, inverted, reverse contrast)
    # ----------------------- TOP ----------------------
    print(model_name)
    acc_top_u, acc_top_i, acc_top_rc = useful_stats_top_bottom[model_name]['top']['acc_u'], useful_stats_top_bottom[model_name]['top']['acc_i'], useful_stats_top_bottom[model_name]['top']['acc_rc']
    acc_bottom_u, acc_bottom_i, acc_bottom_rc = useful_stats_top_bottom[model_name]['bottom']['acc_u'], useful_stats_top_bottom[model_name]['bottom']['acc_i'], useful_stats_top_bottom[model_name]['bottom']['acc_rc']

    top_image_paths_u, top_image_paths_i, top_image_paths_rc  = [get_image_path(idx) for idx, _ in acc_top_u], [get_image_path(idx) for idx, _ in acc_top_i], [get_image_path(idx) for idx, _ in acc_top_rc]
    bottom_image_paths_u, bottom_image_paths_i, bottom_image_paths_rc  = [get_image_path(idx) for idx, _ in acc_bottom_u], [get_image_path(idx) for idx, _ in acc_bottom_i], [get_image_path(idx) for idx, _ in acc_bottom_rc]

    acc_top_u_concat, acc_top_i_concat, acc_top_rc_concat = concatenate_images(top_image_paths_u), concatenate_images(top_image_paths_i), concatenate_images(top_image_paths_rc)
    acc_bottom_u_concat, acc_bottom_i_concat, acc_bottom_rc_concat = concatenate_images(bottom_image_paths_u), concatenate_images(bottom_image_paths_i), concatenate_images(bottom_image_paths_rc)
    # upright
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_top_u_concat)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths_u)} upright for {model_name}", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths_u, acc_top_u)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_top_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # inverted
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_top_i_concat)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths_i)} inverted for {model_name}", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths_i, acc_top_i)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_top_i_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # reverse contrast
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_top_rc_concat)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths_rc)} reverse contrast for {model_name}", fontsize=15)

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths_rc, acc_top_rc)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_top_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    #-------------- BOTTOM -------------------
    # upright
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_bottom_u_concat)
    plt.axis('off')
    plt.title(f"Bottom {len(bottom_image_paths_u)} upright for {model_name}", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths_u, acc_bottom_u)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_bottom_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # inverted
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_bottom_i_concat)
    plt.axis('off')
    plt.title(f"Bottom {len(bottom_image_paths_i)} inverted for {model_name}", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths_i, acc_bottom_i)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_bottom_i_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # reverse contrast
    plt.figure(figsize=(8, 8))
    plt.imshow(acc_bottom_rc_concat)
    plt.axis('off')
    plt.title(f"Bottom {len(bottom_image_paths_rc)} reverse contrast for {model_name}", fontsize=15)

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths_rc, acc_bottom_rc)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, acc_bottom_u_concat.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()



In [ ]:
np.concatenate((elias_acc, neptune_acc))[acc_top_idx_u]

In [ ]:
np.concatenate((elias_acc, neptune_acc))[acc_top_idx_u]

In [ ]:
combined_idx_u

In [ ]:
combined_acc

In [ ]:
combined_acc_u

In [ ]:
acc_top_idx

In [ ]:
combined_idx[acc_top_idx]

In [ ]:
np.concatenate((elias_acc, neptune_acc))[acc_top_idx]

In [ ]:
## vbsl151 -- Highest and lowest accuracy per model 
useful_stats_top_bottom = {}z
data_root = '/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/'
for model_name in model_names_list :
    useful_stats_top_bottom[model_name] = {
        'top' : {
            'acc' : [[],[]],
            'human_i1' : [[],[]],
            'marmoset_i1' : [[],[]],
        },
        'bottom'  : {
            'acc' : [[],[]],
            'human_i1' : [[],[]],
            'marmoset_i1' : [[],[]],
        }
    }

    elias_acc = useful_stats[data_root][model_name]['acc_per_img'][0]
    neptune_acc = useful_stats[data_root][model_name]['acc_per_img'][1]
    combined_acc = np.concatenate((elias_acc, neptune_acc))

    acc_top_idx, acc_bottom_idx = np.argsort(combined_acc)[-15:][::-1], np.argsort(combined_acc)[:15]
    acc_top_val, acc_bottom_val = combined_acc[acc_top_idx], combined_acc[acc_bottom_idx]
    acc_top, acc_bottom = list(zip(acc_top_idx, acc_top_val)), list(zip(acc_bottom_idx, acc_bottom_val))

    useful_stats_top_bottom[model_name]['top']['acc'] = acc_top
    useful_stats_top_bottom[model_name]['bottom']['acc'] = acc_bottom

from PIL import Image as PILImage

def concatenate_images(image_paths):
    images = [PILImage.open(img_path) for img_path in image_paths]
    widths, heights = zip(*(img.size for img in images))

    total_width = sum(widths)
    max_height = max(heights)

    concatenated_image = PILImage.new('RGB', (total_width, max_height))

    x_offset = 0
    for img in images:
        concatenated_image.paste(img, (x_offset, 0))
        x_offset += img.width

    return concatenated_image

# Function to get the image paths based on the index
def get_image_path(idx):
    if idx < 151:
        return f"/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_elias/trialnum{idx}.png"
    else:
        return f"/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/20211011_Var6vbsl_set0_im151_neptune/trialnum{idx-151}.png"

# Displaying the images
for model_name in model_names_list:
    # Get the top 10 and bottom 10 image paths
    acc_top = useful_stats_top_bottom[model_name]['top']['acc']
    acc_bottom = useful_stats_top_bottom[model_name]['bottom']['acc']

    top_image_paths = [get_image_path(idx) for idx, _ in acc_top]
    bottom_image_paths = [get_image_path(idx) for idx, _ in acc_bottom]

    # Concatenate the top 10 images
    top_concatenated_image = concatenate_images(top_image_paths)
    plt.figure(figsize=(25, 8))
    plt.imshow(top_concatenated_image)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths)} images for {model_name}")

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths, acc_top)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, top_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # Concatenate the bottom 10 images
    bottom_concatenated_image = concatenate_images(bottom_image_paths)
    plt.figure(figsize=(25, 8))
    plt.imshow(bottom_concatenated_image)
    plt.axis('off')
    plt.title(f"Bottom {len(top_image_paths)} images for {model_name}")

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths, acc_bottom)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, bottom_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()


In [ ]:
top_image_paths

In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/']['rn50_preIN_texture_sizeVar_best']['i1_corr_diff_per_img'][0][228]

In [ ]:
useful_stats['/mnt/smb/locker/issa-locker/users/Josh/data/face_data/vbsl_151/']['rn50_preIN_texture_sizeVar_best']['i1'][1][228-151]

In [ ]:
human_i1[228]

In [ ]:
for model_name in model_names_list :
    print(useful_stats_top_bottom[model_name]['top']['human_i1'][:10])


In [ ]:
## vbsl151 -- Highest and lowest correlation with human i1 

for model_name in model_names_list :
    # elias_acc = useful_stats[data_root][model_name]['acc_per_img'][0]
    # neptune_acc = useful_stats[data_root][model_name]['acc_per_img'][1]
    combined_human_i1 = useful_stats[data_root][model_name]['i1_corr_diff_per_img'][0]
    combined_marmoset_i1 = useful_stats[data_root][model_name]['i1_corr_diff_per_img'][1]

    human_i1_top_idx, human_i1_bottom_idx = np.argsort(combined_human_i1)[-15:][::-1], np.argsort(combined_human_i1)[:15]
    human_i1_top_val, human_i1_bottom_val = combined_human_i1[human_i1_top_idx], combined_human_i1[human_i1_bottom_idx]
    human_i1_top, human_i1_bottom = list(zip(human_i1_top_idx, human_i1_top_val)), list(zip(human_i1_bottom_idx, human_i1_bottom_val))

    marmoset_i1_top_idx, marmoset_i1_bottom_idx = np.argsort(combined_marmoset_i1)[-15:][::-1], np.argsort(combined_marmoset_i1)[:15]
    marmoset_i1_top_val, marmoset_i1_bottom_val = combined_marmoset_i1[marmoset_i1_top_idx], combined_marmoset_i1[marmoset_i1_bottom_idx]
    marmoset_i1_top, marmoset_i1_bottom = list(zip(marmoset_i1_top_idx, marmoset_i1_top_val)), list(zip(marmoset_i1_bottom_idx, marmoset_i1_bottom_val))

    useful_stats_top_bottom[model_name]['top']['human_i1'] = human_i1_top
    useful_stats_top_bottom[model_name]['bottom']['human_i1'] = human_i1_bottom

    useful_stats_top_bottom[model_name]['top']['marmoset_i1'] = marmoset_i1_top
    useful_stats_top_bottom[model_name]['bottom']['marmoset_i1'] = marmoset_i1_bottom

# Displaying the images
for model_name in model_names_list:
    # Get the top 10 and bottom 10 image paths
    human_i1_top_model = useful_stats_top_bottom[model_name]['top']['human_i1']
    human_i1_bottom_model = useful_stats_top_bottom[model_name]['bottom']['human_i1']

    top_image_paths = [get_image_path(idx) for idx, _ in human_i1_top_model]
    bottom_image_paths = [get_image_path(idx) for idx, _ in human_i1_bottom_model]

    # Concatenate the top 10 images
    top_concatenated_image = concatenate_images(top_image_paths)
    plt.figure(figsize=(25, 8))
    plt.imshow(top_concatenated_image)
    plt.axis('off')
    plt.title(f"Top {len(top_image_paths)} images with human i1 corr, for {model_name}")

    # Add accuracy values below each image for top images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(top_image_paths, human_i1_top_model)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, top_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()

    # Concatenate the bottom 10 images
    bottom_concatenated_image = concatenate_images(bottom_image_paths)
    plt.figure(figsize=(25, 8))
    plt.imshow(bottom_concatenated_image)
    plt.axis('off')
    plt.title(f"Bottom {len(top_image_paths)} images with human i1 corr, for {model_name}")

    # Add accuracy values below each image for bottom images
    x_offset = 0
    for idx, (image_path, (_, acc)) in enumerate(zip(bottom_image_paths, human_i1_bottom_model)):
        img_width = PILImage.open(image_path).width
        plt.text(x_offset + img_width / 2, bottom_concatenated_image.height + 25, f"{acc:.2f}", ha='center', fontsize=10)
        x_offset += img_width

    plt.show()



In [ ]:
useful_statstop_images

In [ ]:
NameError: name 'PILImage' is not defined


In [ ]:
# Accuracy by layer 
layers = ["ResNet50", "Layer1"]
plt.figure(figsize=(10, 6))
# Convert to numpy arrays (if needed)
acc_list = np.array(acc_list)
acc_std_list = np.array(acc_std_list)
plt.plot(layers, acc_list, marker='o', label='accuracy')
plt.fill_between(layers, acc_list - acc_std_list, acc_list + acc_std_list, color='blue', alpha=0.2)
plt.xlabel("layer")
plt.ylabel("accuracy")
plt.title("ResNet50 from scratch, accuracy by layer", fontsize=16)
# plt.errorbar(["ResNet50", "Layer1"], acc_list, yerr=acc_std_list)
# # plt.errorbar(["0.5", "0.25", "0.1", "0.05"], i1_corr_human_list, yerr=i1_corr_std_human_list)
# plt.xlabel("ResNet50 Layer")
# plt.ylabel("i1 corr")
plt.legend()
plt.show()

In [ ]:
# plt.axhline(y=0.444472, label='full 100k dp', color='red')
plt.errorbar(["0.5", "0.25", "0.1", "0.05"], acc_list, yerr=acc_std_list)
# plt.errorbar(["0.5", "0.25", "0.1", "0.05"], i1_corr_human_list, yerr=i1_corr_std_human_list)
plt.xlabel("subset ratio")
plt.ylabel("i1 corr")
plt.legend()
plt.title("training size ablation for vbsl100k")

In [ ]:
# ## subset testing for vbsl151 / no recording

# # 'resnet50', 'alexnet', 'vgg16'
# # 'resnet50', 'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft'
# # 'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl'
# # 'rn50_preIN_notexture_model_best', 'rn50_preIN_texture_model_best'
# # 'rn50_preIN_notexture_sizeVar_best'
# # 'rn50_preIN_notexture_sizeVar_2losses'

# bio_i1s_list = [human_vbsl151, AJ_vbsl151]
# data_root = '../../data/face_data/vbsl_151/'
# # model_name_list = list(useful_stats[data_root].keys())
# model_name_list = ['rn50_preIN_notexture_sizeVar_best']

# num_rep = 1000
# subset_list = [[0,1,2,3,4,5], [0,2], [1,3], [4,5],
#                [0,1,2,3], [0,2,4,5], [1,3,4,5]]
# subset_name_list = ['full', 'normal_face', 'inverted_face', 'reverse_contrast', 
#                    '1+2', '1+3', '2+3']

# for model_name in model_name_list:
#     print(model_name)
#     model_stat = useful_stats[data_root][model_name]
#     # get acc group
#     tmp = model_stat['acc_per_img'].reshape((2,151)).mean(0)
#     tmp_group = tmp[:150].reshape(6,25).mean(axis=1)
#     for subset_id, subset in enumerate(subset_list):
#         # get acc
#         print("%s\nacc: %.2f" % (subset_name_list[subset_id], tmp_group[subset].mean()))
        
#         for bio_inedx, bio_i1s in enumerate(bio_i1s_list):
#             # get model i1s
#             i1_dists = model_stat['dist']
#             i1_dists = np.concatenate((i1_dists[0], i1_dists[1]), axis=1)
#             model_i1s = []
#             for rep_index in range(5):
#                 np.random.shuffle(i1_dists)
#                 tmp = np.concatenate([i1_dists[:int(num_rep/2)].mean(0), 
#                                       i1_dists[int(num_rep/2):].mean(0)])
#                 model_i1s.append(np.expand_dims(tmp, axis=0))
#             model_i1s = np.concatenate(model_i1s, axis=0)
            
#             # get bio i1s
#             bio_i1s = bio_i1s.squeeze()
#             bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only
            
#             # get subset for model/bio i1s
#             model_i1s = model_i1s.reshape((5,4,151))[:,:,:150].reshape((5,4,6,25))
#             model_i1s = model_i1s[:,:,subset,:].reshape(5,-1)
#             bio_i1s = bio_i1s.reshape((5,4,151))[:,:,:150].reshape((5,4,6,25))
#             bio_i1s = bio_i1s[:,:,subset,:].reshape(5,-1)
#             # print(model_i1s.shape, bio_i1s.shape)

#             # calc i1 corr
#             i1_corr = []
#             model_internal = []
#             bio_internal = []
#             half_length = int(model_i1s.shape[1]/2)
#             for rep_index in range(5):
#                 corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
#                 corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
#                 corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
#                 corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
#                 # print(corr_1, corr_2, corr_3, corr_4)
#                 i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
#                 model_internal.append(corr_3)
#                 bio_internal.append(corr_4)
#             print("i1 corr: %.2f+-%.2f, model: %.2f, bio: %.2f" % (np.mean(np.array(i1_corr)), 
#                                                           np.std(np.array(i1_corr)),
#                                                          np.mean(np.array(corr_3)),
#                                                          np.mean(np.array(corr_4))))

## consistency between models

In [ ]:
scipy.stats.pearsonr(dict_i1_dists['resnet50-trained-pretrained-vbsl'].mean(axis=1).reshape(-1), 
                    dict_i1_dists['resnet50-trained-scratch-vbsl'].mean(axis=1).reshape(-1))

In [ ]:
dict_i1_dists.keys()

In [ ]:
# calc i1s corr between models
i1_corr_list = []
for name_0 in model_name_list:
    for name_1 in model_name_list:
        # i1
        _i1_corr, _ = scipy.stats.pearsonr(dict_i1_dists[name_0].mean(axis=1).reshape(-1), 
                                        dict_i1_dists[name_1].mean(axis=1).reshape(-1))
        i1_corr_list.append(_i1_corr)

In [ ]:
i1_corr_list = np.array(i1_corr_list).reshape((len(model_name_list), len(model_name_list)))

In [ ]:
fig, ax = plt.subplots(1,1)
img = ax.imshow(i1_corr_list)

plt.imshow(i1_corr_list)
model_name_list_abb = ['alexnet',
                     'vgg16',
                     'resnet50',
                     'resnet50-SIN',
                     'resnet50-SIN-IN',
                     'resnet50-SIN-IN-ft',
                      'trained-IN+vbsl',
                      'trained-vbsl',
                      'basal-notexture',
                      'basal-texture',
                      'basal-notexture-sizeVar']
ax.set_xticks(np.arange(len(model_name_list_abb)))
ax.set_yticks(np.arange(len(model_name_list_abb)))
ax.set_xticklabels(model_name_list_abb, rotation=45)
ax.set_yticklabels(model_name_list_abb, rotation=45)
ax.set_title("correlations between model i1s - vbsl151")
plt.colorbar()

In [ ]:
np.array(dict_scores['simplecnn-trained-vbsl']).mean()

In [ ]:
plt.scatter(sin_i1_corr_list, sin_conf_mat_jsd_list, label="in/sin")
plt.scatter(scratch_i1_corr_list, scratch_conf_mat_jsd_list, label="scratch")
plt.scatter(cross_i1_corr_list, cross_conf_mat_jsd_list, label="cross")

plt.xlabel("i1")
plt.ylabel("jsd on conf mat")
plt.title("JSD on conf mat vs. i1")
plt.legend(loc="best")
plt.show()

In [ ]:
# calc correlation of i1s
scipy.stats.pearsonr(dists.mean(axis=1).reshape(-1), human_data.reshape(-1))
# scipy.stats.pearsonr(np.absolute(dists.mean(axis=1).reshape(-1)), human_data.reshape(-1))

In [ ]:
plt.scatter(dists.mean(axis=1).reshape(-1), human_data.reshape(-1))
plt.show()

# plots

## plots for lunch talk

In [ ]:
# useful_stats = {}
import pickle
with open('useful_stats_penn.pkl', 'rb') as f:
    useful_stats = pickle.load(f)

In [ ]:
# rn50 
plt.figure(figsize=(1.25*3, 6))
plt.bar(['2D Object', 'GOR', 'GFR'], [0.949, 0.72122017, 0.58074166], 
        yerr=[0.0120, 0.02909256, 0.0242997])
plt.ylim([0.5,1.0])
plt.ylabel("accuracy")
plt.title("resnet50 performances")
plt.savefig(f"./figs/rn50_acc_2d.pdf", dpi=500)

In [ ]:
len(model_ReadableNames), len(i1_mean_list), len(i1_std_list), len(color_list)

In [ ]:
useful_stats[data_root]["resnet50"]['i1_corr_upright']

In [ ]:
# plot i1 - all
data_root = '../../data/face_data/vbsl_151/'
index = 0
subject_list = ['human', 'marmoset']
model_group = ['pixel', 'resnet50_layer1'] + \
                ['rn50_preIN_texture_sizeVar_best', 'rn50_preIN_notexture_sizeVar_best',
                    'resnet50-trained-pretrained-vbsl', 'rn50_vbsl-dist-ft_epoch15'] + \
                   ['vggface', 'rn50_FromScratch_texture_sizeVar_best', 'rn50_FromScratch_notexture_sizeVar_best',
                   'resnet50-trained-scratch-vbsl']
model_ReadableNames = ['pixle', 'v1'] + \
                        ['finetuned-Basel', 'finetuned-Basel-NoTexture', 'finetuned-AppleFaces', 'finetuned-AppleFaces-dist'] + \
                          ['FromScratch-vggface', 'FromScratch-Basel', 'FromScratch-Basel-NoTexture', 'FromScratch-AppleFaces'] + \
                          ['Ideal3D-pixel', 'Ideal3D-rn50', 'Ideal3D-rn50-AppleFaces-ft']
color_list = ['tab:blue'] * 2 + ['tab:grey'] * 4 + ['tab:brown'] * 4 + \
            ['tab:olive'] * 3
# read
i1_mean_list = []
i1_std_list = []
for model_name in model_group:
    i1_corr = useful_stats[data_root][model_name]['i1_corr_upright']
    i1_mean_list.append(i1_corr[index][0])
    i1_std_list.append(i1_corr[index][1])
i1_mean_list += [0.413778, 0.272167, 0.484827]
i1_std_list += [0.070366, 0.020872, 0.011055]
    
# plot
plt.figure(figsize=(2*len(model_group), 6))
plt.bar(model_ReadableNames, i1_mean_list, 
        yerr=i1_std_list, color=color_list)
for i, v in enumerate(i1_mean_list):
    plt.text(i-0.1, v+0.015, str(v)[:5])
plt.ylim([-0.1, 0.65])
plt.xticks(rotation=20)
subject = subject_list[index]
plt.title(f"upright AppleFaces: i1 correlation with {subject}")
plt.ylabel("i1 correlation")
data_name = data_root.split("/")[-2]
plt.axhline(y=-0.01589876, color='r', linestyle='-', label="resnet50")
plt.axhline(y=0, color='black', linestyle='-')

plt.savefig(f"./figs/ideal3d_i1_against_all.pdf", dpi=500)
plt.show()

In [ ]:
# useful_stats = {}
import pickle
with open('useful_stats_tuning.pkl', 'rb') as f:
    useful_stats = pickle.load(f)

In [ ]:
useful_stats.keys()

In [ ]:
# finetuning epoches - face epoch1
data_root = '../../data/face_data/vbsl_151/'
model_group = ['resnet50']
for i in range(41):
    model_group.append(f"vbsl50k_step{(i+1)*15}")
    
# read
i1_mean_list = []
acc_mean_list = []
for model_name in model_group:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_mean_list.append(i1_corr[0][0])
    
    acc = useful_stats[data_root][model_name]['acc']
    acc_mean_list.append(acc[0])

xticks = []
for i in range(42):
    xticks.append((i)*15*256/2)
# plot
plt.plot(xticks, i1_mean_list, marker='o', markersize=3, label='i1 correlation')
plt.plot(xticks, acc_mean_list, marker='o', markersize=3, label='acc')
plt.xlabel("# of images")
plt.title("Elias-Neptune fine-tuning, 1 epoch")
plt.ylabel("GFR% | r(human)")
plt.ylim([-0.1,0.9])
plt.legend()
plt.xticks(rotation=20)
plt.savefig(f"./figs/finetuning_epoches_faces_epoch1.pdf")
plt.show()

In [ ]:
# finetuning epoches - object epoch1
data_root = '../../data/face_data/control_exp/objs/'
model_group = ['resnet50']
for i in range(41):
    model_group.append(f"vbsl50kobj_step{(i+1)*15}")
    
# read
i1_mean_list = []
i1_std_list = []
acc_mean_list = []
acc_std_list = []
for model_name in model_group:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_mean_list.append(i1_corr[0][0])
    i1_std_list.append(i1_corr[0][1])
    
    acc = useful_stats[data_root][model_name]['acc']
    acc_mean_list.append(acc[0])
    acc_std_list.append(acc[1])

xticks = []
for i in range(42):
    xticks.append((i)*15*256/2)
# plot
plt.plot(xticks, i1_mean_list, marker='o', markersize=3, label='i1 correlation')
plt.plot(xticks, acc_mean_list, marker='o', markersize=3, label='acc')
plt.xlabel("# of images")
plt.title("Camel-Elephant fine-tuning, 1 epoch")
plt.ylabel("GOR% | r(human)")
plt.ylim([-0.1,0.9])
plt.legend()
plt.xticks(rotation=20)
plt.savefig(f"./figs/finetuning_epoches_obj_epoch1.pdf")
plt.show()

## plots for penn talk

In [ ]:
# useful_stats = {}
import pickle
with open('useful_stats_penn.pkl', 'rb') as f:
    useful_stats = pickle.load(f)
    
# hierachy: [task_dir][model_name][acc/acc_per_img/dist/i1]
# acc [list]: acc over all datapoints and trails -> [acc_mean, acc_std]
# acc_per_img [array]: acc per image averaged over all trails -> shape [2, num_img_per_class]
# dist [array]: all svm distance (note that the second class dist is negated) -> shape [2, num_repetition, num_img_per_class]
# i1_5rep [array]: random split-half i1 with 5 repetition -> shape [5, 2*num_img_per_class]
# i1 [array]: model i1 (the same as dist averaging over trails) -> shape [2, num_img_per_class]
# i1_corr [list]: model i1 correlation with bio systems. For vbsl101, the order is [Bourgeois, Sausage, AJ]. 
# For vbsl151, the order is [Human, AJ]. For each corr, I record [i1_corr_mean, i1_corr_std]

In [ ]:
useful_stats['../../data/face_data/vbsl_151/']['resnet50']['acc'], useful_stats['../../data/face_data/vbsl_151/']['resnet50']['i1_corr']

In [ ]:
useful_stats['../../data/face_data/control_exp/objs/']['resnet50']['acc'], useful_stats['../../data/face_data/control_exp/objs/']['resnet50']['i1_corr']

In [ ]:
# rn50 
plt.figure(figsize=(1.25*len(model_group), 6))
plt.bar(['GOR', 'GFR'], [0.72122017, 0.58074166], 
        yerr=[0.02909256, 0.0242997])
plt.ylim([0.5,0.8])
plt.ylabel("accuracy")
plt.title("resnet50 performances")
plt.savefig(f"./figs/rn50_acc.pdf", dpi=500)

plt.figure(figsize=(1.25*len(model_group), 6))
plt.bar(['GOR', 'GFR'], [0.4222972, -0.009087089], 
        yerr=[0.02151, 0.0045915])
# plt.ylim([0.5,0.8])
plt.ylabel("i1 correlation")
plt.axhline(y=0, color='black')
plt.title("resnet50 i1 correlation with human")
plt.savefig(f"./figs/rn50_i1.pdf", dpi=500)


In [ ]:
useful_stats.keys()

In [ ]:
useful_stats[data_root_list[4]]['resnet50']['acc']

In [ ]:
data_root_list = ['../../data/face_data/vbsl_151/', '../../data/face_data/control_exp/objs/',
                 '../../data/face_data/control_exp/ood/vbsl151_sophie_praneeth/',
                 '../../data/face_data/control_exp/ood/vbsl151_basel_ood_notexture/',
                 '../../data/face_data/control_exp/ood/vbsl151_basel_ood/']
model_group_list = [['rn50_preIN_texture_sizeVar_best', 'rn50_preIN_notexture_sizeVar_best',
                    'resnet50-trained-pretrained-vbsl', 'rn50_vbsl-dist-ft_epoch15'],
                   ['vggface', 'rn50_FromScratch_texture_sizeVar_best', 'rn50_FromScratch_notexture_sizeVar_best',
                   'resnet50-trained-scratch-vbsl'],
                   ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']]
model_ReadableName_list = [['finetuned-Basel', 'finetuned-Basel-NoTexture', 'finetuned-AppleFaces', 'finetuned-AppleFaces-dist'],
                          ['FromScratch-vggface', 'FromScratch-Basel', 'FromScratch-Basel-NoTexture', 'FromScratch-AppleFaces'],
                          ['SIN', 'SIN+IN', 'SIN+IN with IN-finetuned']]
dataset_name_list = ['AppleFaces vbsli*', 'Objects vbsli*', 'OOD-AppleFaces vbsli*',
                    'OOD-Basel-NoTexture vbsli*', 'OOD-Basel vbsli*']
resnet50_acc = [0.5807416666666666, 0.7212201754385963, 
                0.5027109649122807, 0.6554991228070176, 0.6924236842105265]

In [ ]:
for index in range(4, 5):
    print(index)

In [ ]:
# plot acc - all
for index in range(4, 5):
    data_root = data_root_list[index]
    model_group = ['rn50_preIN_texture_sizeVar_best', 'rn50_preIN_notexture_sizeVar_best',
                        'resnet50-trained-pretrained-vbsl', 'rn50_vbsl-dist-ft_epoch15'] + \
                       ['vggface', 'rn50_FromScratch_texture_sizeVar_best', 'rn50_FromScratch_notexture_sizeVar_best',
                       'resnet50-trained-scratch-vbsl'] + \
                       ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']
    model_ReadableNames = ['finetuned-Basel', 'finetuned-Basel-NoTexture', 'finetuned-AppleFaces', 'finetuned-AppleFaces-dist'] + \
                              ['FromScratch-vggface', 'FromScratch-Basel', 'FromScratch-Basel-NoTexture', 'FromScratch-AppleFaces'] + \
                              ['SIN', 'SIN+IN', 'SIN+IN with IN-finetuned']
    color_list = ['tab:grey'] * 4 + ['tab:brown'] * 4 + ['tab:olive'] * 3
    # read numbers
    acc_mean_list = []
    acc_std_list = []
    for model_name in model_group:
        acc = useful_stats[data_root][model_name]['acc']
        acc_mean_list.append(acc[0]*100)
        acc_std_list.append(acc[1]*100)

    # plot
    plt.figure(figsize=(2*len(model_group), 6))
    plt.bar(model_ReadableNames, acc_mean_list, 
            yerr=acc_std_list, color = color_list)

    for i, v in enumerate(acc_mean_list):
        plt.text(i-0.2, v+3, str(v)[:5])

    plt.ylim([45, 100])
    plt.xticks(rotation=10)
    plt.title(dataset_name_list[index] + " accuracy")
    plt.ylabel("accuracy")
    data_name = data_root.split("/")[-2]

    plt.axhline(y=resnet50_acc[index]*100, color='r', linestyle='-', label="resnet50")
    plt.legend()

    plt.savefig(f"./figs/{data_name}_all.pdf", dpi=500)
    plt.show()

In [ ]:
# plot acc - in groups
for index in range(4, 5):
    data_root = data_root_list[index]
    for model_group_index, model_group in enumerate(model_group_list):
        # read numbers
        acc_mean_list = []
        acc_std_list = []
        for model_name in model_group:
            acc = useful_stats[data_root][model_name]['acc']
            acc_mean_list.append(acc[0]*100)
            acc_std_list.append(acc[1]*100)

        # plot
        plt.figure(figsize=(2*len(model_group), 6))
        plt.bar(model_ReadableName_list[model_group_index], acc_mean_list, yerr=acc_std_list)

        for i, v in enumerate(acc_mean_list):
            plt.text(i-0.2, v+3, str(v)[:5])

        plt.ylim([45, 100])
        plt.xticks(rotation=10)
        plt.title(dataset_name_list[index] + " accuracy")
        plt.ylabel("accuracy")
        data_name = data_root.split("/")[-2]

        plt.axhline(y=resnet50_acc[index]*100, color='r', linestyle='-', label="resnet50")
        plt.legend()

        plt.savefig(f"./figs/{data_name}_group{model_group_index}.pdf", dpi=500)
        plt.show()

In [ ]:
useful_stats['../../data/face_data/vbsl_151/']['resnet50']['i1_corr']

In [ ]:
# plot i1 - in groups
data_root = '../../data/face_data/vbsl_151/'
index = 0
subject_list = ['human', 'marmoset']
for model_group_index, model_group in enumerate(model_group_list):
    # read
    i1_mean_list = []
    i1_std_list = []
    for model_name in model_group:
        i1_corr = useful_stats[data_root][model_name]['i1_corr']
        i1_mean_list.append(i1_corr[index][0])
        i1_std_list.append(i1_corr[index][1])
    
    # plot
    plt.figure(figsize=(2*len(model_group), 6))
    plt.bar(model_ReadableName_list[model_group_index], 
            i1_mean_list, yerr=i1_std_list)
    for i, v in enumerate(i1_mean_list):
        plt.text(i-0.1, v+0.015, str(v)[:5])
    plt.ylim([0, 0.6])
    plt.xticks(rotation=10)
    subject = subject_list[index]
    plt.title(f"AppleFaces: i1 correlation with {subject}")
    plt.ylabel("i1 correlation")
    data_name = data_root.split("/")[-2]
    
    plt.axhline(y=0.009087089407458513, color='r', linestyle='-', label="resnet50")
    # plt.legend(bbox_to_anchor=[1.2, 1])
    
    plt.savefig(f"./figs/{data_name}_group{model_group_index}_{subject}i1.pdf", dpi=500)
    plt.show()

In [ ]:
# plot i1 - all
data_root = '../../data/face_data/vbsl_151/'
index = 0
subject_list = ['human', 'marmoset']
model_group = ['rn50_preIN_texture_sizeVar_best', 'rn50_preIN_notexture_sizeVar_best',
                    'resnet50-trained-pretrained-vbsl', 'rn50_vbsl-dist-ft_epoch15'] + \
                   ['vggface', 'rn50_FromScratch_texture_sizeVar_best', 'rn50_FromScratch_notexture_sizeVar_best',
                   'resnet50-trained-scratch-vbsl'] + \
                   ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']
model_ReadableNames = ['finetuned-Basel', 'finetuned-Basel-NoTexture', 'finetuned-AppleFaces', 'finetuned-AppleFaces-dist'] + \
                          ['FromScratch-vggface', 'FromScratch-Basel', 'FromScratch-Basel-NoTexture', 'FromScratch-AppleFaces'] + \
                          ['SIN', 'SIN+IN', 'SIN+IN with IN-finetuned']
color_list = ['tab:grey'] * 4 + ['tab:brown'] * 4 + ['tab:olive'] * 3
# read
i1_mean_list = []
i1_std_list = []
for model_name in model_group:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_mean_list.append(i1_corr[index][0])
    i1_std_list.append(i1_corr[index][1])

# plot
plt.figure(figsize=(2*len(model_group), 6))
plt.bar(model_ReadableNames, i1_mean_list, 
        yerr=i1_std_list, color=color_list)
for i, v in enumerate(i1_mean_list):
    plt.text(i-0.1, v+0.015, str(v)[:5])
plt.ylim([0, 0.6])
plt.xticks(rotation=10)
subject = subject_list[index]
plt.title(f"AppleFaces: i1 correlation with {subject}")
plt.ylabel("i1 correlation")
data_name = data_root.split("/")[-2]
plt.axhline(y=0.009087089407458513, color='r', linestyle='-', label="resnet50")
plt.savefig(f"./figs/{data_name}_all_{subject}i1.pdf", dpi=500)
plt.show()

plt.show()

In [ ]:
# finetuning epoches - face
data_root = '../../data/face_data/vbsl_151/'
model_group = ['resnet50']
num_epoches = 12
for i in range(num_epoches):
    model_group.append(f'PennTuning-face-epoch{i}')
    # model_names_list.append(f'PennTuning-obj-epoch{i}')

# read
i1_mean_list = []
i1_std_list = []
acc_mean_list = []
acc_std_list = []
for model_name in model_group:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_mean_list.append(i1_corr[0][0])
    i1_std_list.append(i1_corr[0][1])
    
    acc = useful_stats[data_root][model_name]['acc']
    acc_mean_list.append(acc[0])
    acc_std_list.append(acc[1])
    
# plot
plt.errorbar(np.arange(num_epoches+1), i1_mean_list, i1_std_list, label='i1 correlation')
plt.errorbar(np.arange(num_epoches+1), acc_mean_list, acc_std_list, label='acc')
plt.xlabel("finetuning epoches")
plt.title("finetuning for AppleFaces")
plt.ylim([-0.1,0.9])
plt.legend()
plt.savefig(f"./figs/finetuning_epoches_faces.pdf")
plt.show()

In [ ]:
# finetuning epoches - obj
data_root = '../../data/face_data/control_exp/objs/'
model_group = ['resnet50']
num_epoches = 12
for i in range(num_epoches):
    model_group.append(f'PennTuning-obj-epoch{i}')

# read
i1_mean_list = []
i1_std_list = []
acc_mean_list = []
acc_std_list = []
for model_name in model_group:
    i1_corr = useful_stats[data_root][model_name]['i1_corr']
    i1_mean_list.append(i1_corr[0][0])
    i1_std_list.append(i1_corr[0][1])
    
    acc = useful_stats[data_root][model_name]['acc']
    acc_mean_list.append(acc[0])
    acc_std_list.append(acc[1])
# plot
plt.errorbar(np.arange(num_epoches+1), i1_mean_list, i1_std_list, label='i1 correlation')
plt.errorbar(np.arange(num_epoches+1), acc_mean_list, acc_std_list, label='acc')
plt.xlabel("finetuning epoches")
plt.title("finetuning for Objects")
plt.ylim([-0.1,0.9])
plt.legend(loc='lower right')
plt.savefig(f"./figs/finetuning_epoches_objects.pdf")
plt.show()

In [ ]:
useful_stats_tuning

In [ ]:
# reg analysis - bar plot - in groups
index = 0
data_root = data_root_list[index]
r2_list = [[0.07792108787342954, 0.2653578840672663, 0.4046336447971614, 0.452],
           [0.02826135053361982, 0.025643561490320668, 0.2185898009488952, 0.49304382365514515],
           [0.070180975320918, 0.006955316238228294, 0.03518525749237562]]

for model_group_index, model_group in enumerate(model_group_list):
    # read numbers
    cur_r2_list = r2_list[model_group_index]
    
    # plot
    plt.figure(figsize=(2*len(model_group), 6))
    plt.bar(model_ReadableName_list[model_group_index], cur_r2_list)
    plt.axhline(y=0.5687079406432649, label='human', color='red')
    plt.axhline(y=0.28832485400284535, label='marmoset', color='green')
    plt.axhline(y=0.0149, label='resnet50', color='yellow')

    for i, v in enumerate(cur_r2_list):
        plt.text(i-0.1, v+0.01, str(v)[:5])
        
    plt.ylim([0, 0.6])
    plt.xticks(rotation=10)
    plt.title("Regression of latent variables")
    plt.ylabel("r2")
    data_name = data_root.split("/")[-2]
    plt.legend(bbox_to_anchor=[1,1.01])
    plt.savefig(f"./figs/reg_analysis_r2_group{model_group_index}.pdf", dpi=500)
    plt.show()

In [ ]:
# reg analysis - bar plot - all
index = 0
data_root = data_root_list[index]
r2_list = [0.07792108787342954, 0.2653578840672663, 0.4046336447971614, 0.452] +\
           [0.02826135053361982, 0.025643561490320668, 0.2185898009488952, 0.49304382365514515] +\
           [0.070180975320918, 0.006955316238228294, 0.03518525749237562]
color_list = ['tab:grey'] * 4 + ['tab:brown'] * 4 + ['tab:olive'] * 3
model_ReadableNames = ['finetuned-Basel', 'finetuned-Basel-NoTexture', 'finetuned-AppleFaces', 'finetuned-AppleFaces-dist'] + \
                          ['FromScratch-vggface', 'FromScratch-Basel', 'FromScratch-Basel-NoTexture', 'FromScratch-AppleFaces'] + \
                          ['SIN', 'SIN+IN', 'SIN+IN with IN-finetuned']
# read numbers
cur_r2_list = r2_list
# plot
plt.figure(figsize=(2*len(model_ReadableNames), 6))
plt.bar(model_ReadableNames, cur_r2_list, color = color_list)
plt.axhline(y=0.5687079406432649, label='human', color='red')
plt.axhline(y=0.28832485400284535, label='marmoset', color='green')
plt.axhline(y=0.0149, label='resnet50', color='yellow')
for i, v in enumerate(cur_r2_list):
    plt.text(i-0.1, v+0.01, str(v)[:5])

plt.ylim([0, 0.6])
plt.xticks(rotation=10)
plt.title("Regression of latent variables")
plt.ylabel("r2")
data_name = data_root.split("/")[-2]
plt.legend(bbox_to_anchor=[1,1.01])

plt.savefig(f"./figs/reg_analysis_r2_all.pdf", dpi=500)
plt.show()

In [ ]:
# reg analysis - significant regressor
from matplotlib import colors
regressors_list = ["inversion", "contrast", "lighting", 
                    "rotation_x", "rotation_y", "size"]
data = np.array([[2, 2, 0, 2, 0, 2],
                 [2, 1, 0, 2, 2, 0]])

# create discrete colormap
cmap = colors.ListedColormap(['black', 'rosybrown', 'red'])
bounds = [0, 0.5, 1.5, 2]
norm = colors.BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(data.shape[1], data.shape[0]))

ax.set_xticks(np.arange(data.shape[1])+0.5, minor=False)
ax.set_yticks(np.arange(data.shape[0])+0.5, minor=False)
ax.set_xticklabels(regressors_list, minor=False)
ax.set_yticklabels(['human', 'marmoset'], minor=False)

ax.pcolor(data, edgecolors='white', linestyle= 'dashed', linewidths=1, 
          cmap=cmap, norm=norm)
# ax.imshow(data, cmap=cmap, norm=norm)
# ax.imshow(data, cmap="Greens")

# ax.grid(which="minor", axis='both', linewidth=2)
# ax.tick_params(which="minor", size=0)
# ax.grid(which='minor', axis='both', linestyle='-', color='k', linewidth=2)
plt.xlim((0, data.shape[1]))
plt.title("Regression analysis of latent variables vs. i1")
plt.savefig(f"./figs/reg_analysis_LatentVar_HumanMarmoset.pdf")
plt.show()

In [ ]:
# reg analysis - significant regressor
from matplotlib import colors
regressors_list = ["inversion", "contrast", "lighting", 
                    "rotation_x", "rotation_y", "size"]
data = np.array([[2, 2, 0, 2, 0, 2],
                  [1, 2, 0, 0, 0, 2],
                  [2, 2, 0, 0, 0, 0],
                  [2, 2, 0, 0, 0, 0],
                  [2, 2, 0, 2, 0, 2],
                  [0, 0, 0, 0, 0, 1],
                  [0, 0, 0, 0, 0, 0],
                  [2, 2, 0, 0, 0, 0],
                  [2, 2, 0, 2, 0, 0],
                  [2, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 1, 0]][::-1])
model_group = ['rn50_preIN_texture_sizeVar_best',
                  'rn50_preIN_notexture_sizeVar_best',
                  'resnet50-trained-pretrained-vbsl',
                  'rn50_vbsl-dist-ft_epoch15'] +\
                 ['vggface',
                  'rn50_FromScratch_texture_sizeVar_best',
                  'rn50_FromScratch_notexture_sizeVar_best',
                  'resnet50-trained-scratch-vbsl'] +\
                 ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']
# create discrete colormap
cmap = colors.ListedColormap(['black', 'rosybrown', 'red'])
bounds = [0, 0.5, 1.5, 2]
norm = colors.BoundaryNorm(bounds, cmap.N)

fig, ax = plt.subplots(figsize=(data.shape[1], data.shape[0]))

ax.set_xticks(np.arange(data.shape[1])+0.5, minor=False)
ax.set_yticks(np.arange(data.shape[0])+0.5, minor=False)
ax.set_xticklabels(regressors_list, minor=False)
ax.set_yticklabels((['human'] + model_ReadableNames)[::-1], minor=False)

ax.pcolor(data, edgecolors='white', linestyle= 'dashed', linewidths=1, 
          cmap=cmap, norm=norm)
# ax.imshow(data, cmap=cmap, norm=norm)
# ax.imshow(data, cmap="Greens")

# ax.grid(which="minor", axis='both', linewidth=2)
# ax.tick_params(which="minor", size=0)
# ax.grid(which='minor', axis='both', linestyle='-', color='k', linewidth=2)
plt.xlim((0, data.shape[1]))
plt.title("Regression analysis of latent variables vs. i1")
plt.savefig(f"./figs/reg_analysis_LatentVar_HumanModels.pdf")
plt.show()

## others

In [ ]:
# 
xx = [1, 3, 6, 9, 12, 15]
# acc_list = [0.7947, 0.7538, 0.7464, 0.7464, 0.7428, 0.7791, 0.7550]
means = acc_list 
plt.plot(xx, means, label = 'accuracy')
for i, v in enumerate(means):
    plt.text(xx[i]-0.20, v+0, "%.2f"%(v))

means = i1_corr_human_list
plt.plot(xx, means, label = 'i1 corr with human')
for i, v in enumerate(means):
    plt.text(xx[i]-0.20, v+0, "%.2f"%(v))
    
means = i1_corr_AJ_list
plt.plot(xx, means, label = 'i1 corr with AJ')
for i, v in enumerate(means):
    plt.text(xx[i]-0.20, v+0, "%.2f"%(v))

plt.title("ft epochs tuning for basal-pretrained+vbsl50k-finetuned")
plt.xlabel("ft epochs")
plt.legend(bbox_to_anchor=[1.02,1])
plt.ylabel("acc")
plt.show()

In [ ]:
plt.plot(['10', '100', '1000', '10000'], [0.6411, 0.6284, 0.6141, 0.6428], 
         c='blue', label='acc - with depth map pred')
plt.axhline(y=0.63, 
            c='blue', label='acc - w/o depth map pred')
plt.plot(['10', '100', '1000', '10000'], [0.3089, 0.2331, 0.3822, 0.2841],
         c='red', label='i1 corr - with depth map pred')
plt.axhline(y=0.32, 
            c='red', label='i1 corr - w/o depth map pred')
plt.legend()
plt.show()

### RSA

In [ ]:
import numpy, rsatoolbox
data = rsatoolbox.data.Dataset(numpy.random.rand(10, 5))
rdms = rsatoolbox.rdm.calc_rdm(data)
rsatoolbox.vis.show_rdm(rdms)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
rdms.dissimilarities

In [ ]:
# vbsp 101
data_root = '../../data/face_data/vbsp_101/'
exp_name_0 = '20210811_Var6vbsp_set0_im101_elias'
exp_name_1 = '20210811_Var6vb_same_as_elias_sp_set0_im101_neptune'
filename_postfix = ''

model_name_list = ['resnet50', 'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft',
                   'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl',
                   'simplecnn-trained-vbsl']

# model_name_list = ['resnet50', 'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']
# 'resnet50-trained-pretrained-vbsl'
# model_name_list = ['resnet50-trained-scratch-vbsl', 'simplecnn-trained-vbsl']

rdms = {}
for model_name in model_name_list:
    # read feats
    input_1 = torch.load(os.path.join(data_root,
                                  exp_name_0+'_'+model_name+filename_postfix+'.pth'))
    input_2 = torch.load(os.path.join(data_root,
                                      exp_name_1+'_'+model_name+filename_postfix+'.pth'))
    inputs = np.concatenate((input_1, input_2), axis=0)
    data = rsatoolbox.data.Dataset(inputs)
    rdms[model_name] = rsatoolbox.rdm.calc_rdm(data)

In [ ]:
corr_rdms = []
for model_name_0 in model_name_list:
    _corr_rdms = []
    for model_name_1 in model_name_list:
        _corr = scipy.stats.pearsonr(rdms[model_name_0].dissimilarities.squeeze(), 
                             rdms[model_name_1].dissimilarities.squeeze())
        _corr_rdms.append(_corr[0])
    corr_rdms.append(_corr_rdms)

In [ ]:
import seaborn as sns

ax = plt.imshow(corr_rdms, cmap='hot', interpolation='nearest')
plt.colorbar(label="corr")
plt.show()

In [ ]:
rdms.keys()

In [ ]:
rsatoolbox.vis.show_rdm(rdms['simplecnn-trained-vbsl'])
plt.show()

In [ ]:
rsatoolbox.vis.show_rdm(rdms['resnet50-trained-pretrained-vbsl'])
plt.show()

## train/test on diff imgs

In [ ]:
### prepare training data

# # vbsl 101
# data_root = '../../data/face_data/vbsl_101/'
# exp_name_0 = '20210920_Var6vbsl_set0_im101_elias'
# exp_name_1 = '20210920_Var6vbsl_set0_im101_neptune'

# vbsl 200
data_root = '../../data/face_data/vbsl_200_new_bg/'
exp_name_0 = '2022105_Var6vbsl_set0_im200_elias'
exp_name_1 = '2022105_Var6vbsl_set0_im200_neptune'
filename_postfix = ''

# model_name_list = ['resnet50', 'alexnet', 'vgg16']
# model_name_list = ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']
# model_name_list = ['resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'simplecnn-trained-vbsl']
# model_name_list = ['rn50_preIN_notexture_model_best', 'rn50_preIN_texture_model_best']
# model_name_list = ['rn50_preIN_notexture_sizeVar_best']
# model_name_list = ['rn50_preIN_notexture_sizeVar_2losses', 'rn50_preIN_notexture_sizeVar_onlyDM']
# model_name_list = ['rn50_preIN_notexture_sizeVar_onlyDM']

model_name = 'resnet50-trained-scratch-vbsl'

# read feats
train_input_1 = torch.load(os.path.join(data_root,
                              exp_name_0+'_'+model_name+filename_postfix+'.pth'))
train_input_2 = torch.load(os.path.join(data_root,
                              exp_name_1+'_'+model_name+filename_postfix+'.pth'))
train_input_1 = train_input_1.reshape(len(train_input_1), -1)
train_input_2 = train_input_2.reshape(len(train_input_2), -1)
print(train_input_1.shape, train_input_2.shape)
# rebalance: shuffle and remove
torch.manual_seed(7)
# prepare data
train_output_1 = np.ones(len(train_input_1))
train_output_2 = np.zeros(len(train_input_2))
train_inputs = np.concatenate((train_input_1, train_input_2), axis=0)
train_outputs = np.concatenate((train_output_1, train_output_2), axis=0)

In [ ]:
### prepare test data

# vbsl 151
data_root = '../../data/face_data/vbsl_151/'
exp_name_0 = '20211011_Var6vbsl_set0_im151_elias'
exp_name_1 = '20211011_Var6vbsl_set0_im151_neptune'

# read feats
test_input_1 = torch.load(os.path.join(data_root,
                              exp_name_0+'_'+model_name+filename_postfix+'.pth'))
test_input_2 = torch.load(os.path.join(data_root,
                                  exp_name_1+'_'+model_name+filename_postfix+'.pth'))
test_input_1 = test_input_1.reshape(len(test_input_1), -1)
test_input_2 = test_input_2.reshape(len(test_input_2), -1)
print(test_input_1.shape, test_input_2.shape)
test_output_1 = np.ones(len(test_input_1))
test_output_2 = np.zeros(len(test_input_2))
test_inputs = np.concatenate((test_input_1, test_input_2), axis=0)
test_outputs = np.concatenate((test_output_1, test_output_2), axis=0)

In [ ]:
# train & test classifiers

num_rep = 100
acc_per_img = np.nan * np.ones((num_rep, len(test_outputs))) 
dists = np.nan * np.ones((num_rep, len(test_outputs))) 

for rep_index in tqdm(range(num_rep)):
    cv = StratifiedKFold(n_splits=2, shuffle=True)
    scores = []

    # iter through splits of cv
    for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
        # split data
        X_train = np.concatenate((train_input_1[train_index], train_input_2[train_index]), axis=0)
        y_train = np.concatenate((train_output_1[train_index], train_output_2[train_index]), axis=0)
        
        # fit model
        clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                             tol=1e-4,
                             fit_intercept=True,
                             C=1.0,
                            max_iter = 20000)
        clf.fit(X_train, y_train)
        # record score & dist - part 1
        y_predict = clf.predict(test_input_1)
        _acc_per_img = (y_predict == test_output_1).astype('float32')
        acc_per_img[rep_index][:len(test_input_1)] = _acc_per_img
        _dist = clf.decision_function(test_input_1)
        dists[rep_index][:len(test_input_1)] = _dist
        # record score & dist - part 2
        y_predict = clf.predict(test_input_2)
        _acc_per_img = (y_predict == test_output_2).astype('float32')
        acc_per_img[rep_index][len(test_input_2):] = _acc_per_img
        _dist = clf.decision_function(test_input_2)
        dists[rep_index][len(test_input_2):] = _dist * (-1) # negate for diff sign
        
        # only use half of the training set 
        break
acc_per_img = acc_per_img.mean(0)

In [ ]:
# get stats -> vbsl151 subset analysis

subset_list = [[0,2], [1,3], [4,5],
               [0,1,2,3], [0,2,4,5], [1,3,4,5], [0,1,2,3,4,5]]
subset_name_list = ['normal_face', 'inverted_face', 'reverse_contrast', 
                   '1+2', '1+3', '2+3', 'full']

print(model_name)

for subset_id, subset in enumerate(subset_list):
    # get acc
    tmp = acc_per_img.reshape((2,151)).mean(0)
    tmp_group = tmp[:150].reshape(6,25).mean(axis=1)
    print("%s\nacc: %.2f" % (subset_name_list[subset_id], tmp_group[subset].mean()))
    
    # get bio i1s
    bio_i1s = human_i1s.squeeze()
    # model i1s
    i1_dists = dists
    model_i1s = []
    for rep_index in range(5):
        np.random.shuffle(i1_dists)
        tmp = np.concatenate([i1_dists[:int(num_rep/2)].mean(0), 
                              i1_dists[int(num_rep/2):].mean(0)])
        model_i1s.append(np.expand_dims(tmp, axis=0))
    model_i1s = np.concatenate(model_i1s, axis=0)
    # get subset
    model_i1s = model_i1s.reshape((5,4,151))[:,:,:150].reshape((5,4,6,25))
    model_i1s = model_i1s[:,:,subset,:].reshape(5,-1)
    bio_i1s = bio_i1s.reshape((5,4,151))[:,:,:150].reshape((5,4,6,25))
    bio_i1s = bio_i1s[:,:,subset,:].reshape(5,-1)
    
    # print(model_i1s.shape, bio_i1s.shape)
    
    # calc i1 corr
    i1_corr = []
    model_internal = []
    bio_internal = []
    half_length = int(model_i1s.shape[1]/2)
    for rep_index in range(5):
        corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
        corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
        corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
        corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
        # print(corr_1, corr_2, corr_3, corr_4)
        i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
        model_internal.append(corr_3)
        bio_internal.append(corr_4)
    print("i1 corr: %.2f+-%.2f, model: %.2f, bio: %.2f" % (np.mean(np.array(i1_corr)), 
                                                  np.std(np.array(i1_corr)),
                                                 np.mean(np.array(corr_3)),
                                                 np.mean(np.array(corr_4))))

## train/test on subset

In [ ]:
# vbsl 151
data_root = '../../data/face_data/vbsl_151/'
exp_name_0 = '20211011_Var6vbsl_set0_im151_elias'
exp_name_1 = '20211011_Var6vbsl_set0_im151_neptune'

# # vbsl 151 - obj
# data_root = '../../data/face_data/control_exp/objs/'
# exp_name_0 = '20211011_Var6vbsl_set0_im151_camel'
# exp_name_1 = '20211011_Var6vbsl_set0_im151_elephant'

filename_postfix = ''
num_rep = 100

# model_name_list = ['resnet50', 'alexnet', 'vgg16']
# model_name_list = ['resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft']
# model_name_list = ['resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl', 'simplecnn-trained-vbsl']
# model_name_list = ['rn50_preIN_notexture_model_best', 'rn50_preIN_texture_model_best']
# model_name_list = ['rn50_preIN_notexture_sizeVar_best']
# model_name_list = ['rn50_preIN_notexture_sizeVar_2losses', 'rn50_preIN_notexture_sizeVar_onlyDM']
# model_name_list = ['rn50_preIN_notexture_sizeVar_onlyDM']

model_name_list = ['rn50_preIN_notexture_sizeVar_best', 'resnet50', 
                  'resnet50-trained-pretrained-vbsl', 
                  'rn50_preIN_notexture_model_best']

dict_scores = {}
dict_i1_dists = {}
dict_conf_mat = {}
acc_per_img = {}

subset_list = [[0,2], [1,3], [4,5],
               [0,1,2,3], [0,2,4,5], [1,3,4,5]]
subset_name_list = ['normal_face', 'inverted_face', 'reverse_contrast', 
                   '1+2', '1+3', '2+3']

for model_name in model_name_list:
    # iter over subsets: 
    for subset_index in range(len(subset_list)):
        subset = subset_list[subset_index]
        subset_name = subset_name_list[subset_index]
    
        # read feats
        _path = os.path.join(data_root, exp_name_0+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_1 from {_path}")
        input_1 = torch.load(_path)
        _path = os.path.join(data_root, exp_name_1+'_'+model_name+filename_postfix+'.pth')
        print(f"loading input_2 from {_path}")
        input_2 = torch.load(_path)
        input_1 = input_1.reshape(len(input_1), -1)
        input_2 = input_2.reshape(len(input_2), -1)
        print(input_1.shape, input_2.shape)
        # rebalance: shuffle and remove
        torch.manual_seed(7)
        if len(input_1) != len(input_2):
            if len(input_2)>len(input_1):
                input_3 = input_1
                input_1 = input_2
                input_2 = input_3
                del input_3
            idx = torch.randperm(input_1.shape[0])
            input_1 = input_1[idx]
            input_1 = input_1[:len(input_2)]

        print(input_1.shape, input_2.shape)
        # prepare data
        output_1 = np.ones(len(input_1))
        output_2 = np.zeros(len(input_2))
    
        # subsetting input
        input_1 = input_1[:150].reshape((6,25,input_1.shape[1]))
        input_1 = input_1[subset,:].reshape(-1,2048)
        input_2 = input_2[:150].reshape((6,25,input_2.shape[1]))
        input_2 = input_2[subset,:].reshape(-1,2048)
        # subsetting output
        output_1 = output_1[:150].reshape((6,25))
        output_1 = output_1[subset,:].reshape(-1)
        output_2 = output_2[:150].reshape((6,25))
        output_2 = output_2[subset,:].reshape(-1)
        
        inputs = np.concatenate((input_1, input_2), axis=0)
        outputs = np.concatenate((output_1, output_2), axis=0)


        # cross-validation
        # [num_class, num_rep, num_img_per_class]
        # [2, 1000, 101]
        dists = np.nan * np.ones((2, num_rep, len(output_1))) 
        all_scores = []
        acc = np.nan * np.ones((2, num_rep, len(output_1))) 

        # iter num_rep
        conf_mat = np.zeros((2,2))
        for rep_index in tqdm(range(num_rep)):
            cv = StratifiedKFold(n_splits=2, shuffle=True)
            scores = []

            # iter through splits of cv
            for split_index, (train_index, test_index) in enumerate(cv.split(input_1, output_1)):
                # split data
                X_train = np.concatenate((input_1[train_index], input_2[train_index]), axis=0)
                X_test = np.concatenate((input_1[test_index], input_2[test_index]), axis=0)
                y_train = np.concatenate((output_1[train_index], output_2[train_index]), axis=0)
                y_test = np.concatenate((output_1[test_index], output_2[test_index]), axis=0)
                # fit model
                clf = svm.LinearSVC(penalty='l2', loss='hinge', dual=True,
                                     tol=1e-4,
                                     fit_intercept=True,
                                     C=1.0,
                                    max_iter = 20000)
                clf.fit(X_train, y_train)
                # record score
                y_predict = clf.predict(X_test)
                _score = (y_predict == y_test).sum() / len(y_predict)
                scores.append(_score)
                _acc_per_img = (y_predict == y_test).astype('float32')
                acc[0][rep_index][test_index] = _acc_per_img[:len(test_index)]
                acc[1][rep_index][test_index] = _acc_per_img[len(test_index):]
                # record dist
                _class1_dist = clf.decision_function(input_1[test_index])
                dists[0][rep_index][test_index] = _class1_dist
                _class2_dist = clf.decision_function(input_2[test_index])
                dists[1][rep_index][test_index] = _class2_dist * (-1) # negate the dist for second class

            all_scores.append(np.array(scores).mean())

        dists = dists/num_rep
        print(model_name, subset_name, '%.4f+-%.4f'%(np.array(all_scores).mean(), np.array(all_scores).std()))
        
        if model_name not in dict_i1_dists:
            dict_i1_dists[model_name] = {}
        if model_name not in acc_per_img:
            acc_per_img[model_name] = {}
        dict_i1_dists[model_name][subset_name] = dists
        acc_per_img[model_name][subset_name] = acc

In [ ]:
import pickle
useful_stats_subset = {}
useful_stats_subset['dist'] = dict_i1_dists
useful_stats_subset['acc_per_img'] = acc_per_img

with open('useful_stats_subset.pkl', 'wb') as f:
    pickle.dump(useful_stats_subset, f)

In [ ]:
# calc i1 corr

## subset testing for vbsl151 / no recording

# 'resnet50', 'alexnet', 'vgg16'
# 'resnet50', 'resnet50-SIN', 'resnet50-SIN-IN', 'resnet50-SIN-IN-ft'
# 'resnet50-trained-pretrained-vbsl', 'resnet50-trained-scratch-vbsl'
# 'rn50_preIN_notexture_model_best', 'rn50_preIN_texture_model_best'
# 'rn50_preIN_notexture_sizeVar_best'
# 'rn50_preIN_notexture_sizeVar_2losses'

bio_i1s_list = [human_vbsl151]
data_root = '../../data/face_data/vbsl_151/'
# model_name_list = list(useful_stats[data_root].keys())

num_rep = 100
subset_list = [[0,2], [1,3], [4,5],
               [0,1,2,3], [0,2,4,5], [1,3,4,5]]
subset_name_list = ['normal_face', 'inverted_face', 'reverse_contrast', 
                   '1+2', '1+3', '2+3']

for model_name in model_name_list:
    print(model_name)
    
    for subset_id, subset in enumerate(subset_list):
        subset_name = subset_name_list[subset_id]
        # get acc
        print("%s\nacc: %.2f" % (subset_name, useful_stats_subset['acc_per_img'][model_name][subset_name].mean()))
        
        for bio_inedx, bio_i1s in enumerate(bio_i1s_list):
            # get model i1s
            i1_dists = useful_stats_subset['dist'][model_name][subset_name]
            i1_dists = np.concatenate((i1_dists[0], i1_dists[1]), axis=1)
            model_i1s = []
            for rep_index in range(5):
                np.random.shuffle(i1_dists)
                tmp = np.concatenate([i1_dists[:int(num_rep/2)].mean(0), 
                                      i1_dists[int(num_rep/2):].mean(0)])
                model_i1s.append(np.expand_dims(tmp, axis=0))
            model_i1s = np.concatenate(model_i1s, axis=0)
            
            # get bio i1s
            bio_i1s = bio_i1s.squeeze()
            bio_i1s = np.nan_to_num(bio_i1s, nan=4) # for monkey_i1s_3 only
            
            # get subset for model/bio i1s
            # no need to subset model_i1s
            # model_i1s = model_i1s.reshape((5,4,151))[:,:,:150].reshape((5,4,6,25))
            # model_i1s = model_i1s[:,:,subset,:].reshape(5,-1)
            bio_i1s = bio_i1s.reshape((5,4,151))[:,:,:150].reshape((5,4,6,25))
            bio_i1s = bio_i1s[:,:,subset,:].reshape(5,-1)
            # print(model_i1s.shape, bio_i1s.shape)

            # calc i1 corr
            i1_corr = []
            model_internal = []
            bio_internal = []
            half_length = int(model_i1s.shape[1]/2)
            for rep_index in range(5):
                corr_1, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], bio_i1s[rep_index][half_length:])
                corr_2, _ = scipy.stats.pearsonr(model_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                corr_3, _ = scipy.stats.pearsonr(model_i1s[rep_index][:half_length], model_i1s[rep_index][half_length:])
                corr_4, _ = scipy.stats.pearsonr(bio_i1s[rep_index][half_length:], bio_i1s[rep_index][:half_length])
                # print(corr_1, corr_2, corr_3, corr_4)
                i1_corr.append(0.5*(corr_1+corr_2)/np.sqrt(corr_3*corr_4))
                model_internal.append(corr_3)
                bio_internal.append(corr_4)
            print("i1 corr: %.2f+-%.2f, model: %.2f, bio: %.2f\n" % (np.mean(np.array(i1_corr)), 
                                                          np.std(np.array(i1_corr)),
                                                         np.mean(np.array(corr_3)),
                                                         np.mean(np.array(corr_4))))